# KG Edit

In [ ]:
!git clone https://github.com/zjunlp/EasyEdit
%cd EasyEdit
!git checkout f0ec5eb3ef6a09f637fa5344667a78f38fbeb05d
!pip install -r colab_requirements.txt
!pip install qwen-vl-utils
!pip install av
!pip install openai==1.2.0
!pip install zhipuai
!pip install hydra-core -U
!pip install sentence_transformers==3.2.1
!pip install -U datasets
!pip install -q sparqlwrapper


Cloning into 'EasyEdit'...
remote: Enumerating objects: 8523, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 8523 (delta 7), reused 3 (delta 3), pack-reused 8502 (from 2)
Receiving objects: 100% (8523/8523), 85.85 MiB | 42.24 MiB/s, done.
Resolving deltas: 100% (5412/5412), done.
/kaggle/working/EasyEdit
Note: switching to 'f0ec5eb3ef6a09f637fa5344667a78f38fbeb05d'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at f0ec5eb fix bug in notebooks an

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForCausalLM
from SPARQLWrapper import SPARQLWrapper, JSON
import torch
from tqdm.auto import tqdm
import logging
import sys
import time
import re
import json
import os
import os.path as path
import numpy as np
from easyeditor import KnowEditDataset


In [ ]:
torch.set_grad_enabled(False)

logging.basicConfig(
    filename='kgedit.log',
    filemode='w',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger('KGEdit')

handler = logging.FileHandler('kgedit.log')
handler.setLevel(logging.INFO)
formatter = logging.Formatter('[%(asctime)s] %(name)s - %(levelname)s: %(message)s', 
                               datefmt='%H:%M:%S')
handler.setFormatter(formatter)
logger.addHandler(handler)

# logger = logging.getLogger('KGEdit')
# logger.setLevel(logging.INFO)
# handler = logging.StreamHandler(sys.stdout)
# handler.setLevel(logging.INFO)
# formatter = logging.Formatter('[%(asctime)s] %(name)s - %(levelname)s: %(message)s', 
#                               datefmt='%H:%M:%S')
# handler.setFormatter(formatter)
# logger.addHandler(handler)
# logging.getLogger("transformers").setLevel(logging.WARNING)
# logging.getLogger("urllib3").setLevel(logging.WARNING)


In [ ]:
USE_CHAT_TEMPLATE = False
MODEL = 'qwen2.5-1.5b-instruct' # qwen2.5-1.5b-instruct qwen2.5-3b-instruct
if 'instruct' in MODEL:
    USE_CHAT_TEMPLATE = True


In [ ]:
# Load Dataset

# %mkdir data
!huggingface-cli download zjunlp/KnowEdit --repo-type dataset --local-dir /kaggle/working/EasyEdit/data


Fetching 14 files:   0%|                                 | 0/14 [00:00<?, ?it/s]Downloading 'benchmark/ZsRE/ZsRE-test-all.json' to '/kaggle/working/EasyEdit/data/.cache/huggingface/download/benchmark/ZsRE/o5LGPIEg1EjkOuxgs9IOBWsg4QA=.759a1d0c7b87d4d1612f655dbbac4cb8262b2948.incomplete'

wikibio-train-all.json:   0%|                        | 0.00/454k [00:00<?, ?B/s]Downloading 'benchmark/Convsent/blender_train.json' to '/kaggle/working/EasyEdit/data/.cache/huggingface/download/benchmark/Convsent/i-mF4aBv7Lkq4rpoc_Thnmm6zL8=.0ebd6d70e31127e3c93dd2f235bc4963e9b3bf4e6219a96c5aa91beab85aa01c.incomplete'


blender_val.json:   0%|                             | 0.00/1.33M [00:00<?, ?B/s]


wikibio-test-all.json:   0%|                         | 0.00/310k [00:00<?, ?B/s]



.gitattributes: 100%|██████████████████████| 2.38k/2.38k [00:00<00:00, 25.5MB/s]
Download complete. Moving file to /kaggle/working/EasyEdit/data/.gitattributes
Fetching 14 files:   7%|█▊                       | 1/14 [00:00<0

In [ ]:
class KnowledgeGraph:
    def __init__(self, use_wiki=True, endpoint='https://query.wikidata.org/sparql'):
        self.logger = logging.getLogger('KGEdit.KnowledgeGraph')
        self.logger.info('Initialising knowledge graph...')
        self.endpoint = endpoint
        self.local_facts = {}
        self.cache = {}
        self.use_wiki = use_wiki
        self.logger.info('Knowledge graph initialised!')

    def add_fact(self, subject, relation, obj):
        self.logger.info(f'Adding fact: Subject: {subject}, Relation: {relation}, Object: {obj}')
        key = f'{subject.lower()}|{relation.lower()}'
        self.local_facts[key] = obj

    def query(self, subject, relation):
        # Checking local facts first
        self.logger.info(f'Checking local facts! Subject: {subject}, relation: {relation}')
        key = f'{subject.lower()}|{relation.lower()}'
        if key in self.cache:
            self.logger.info(f'Cached fact found: {self.cache[key]}')
            return self.cache[key]
        
        if key in self.local_facts:
            self.logger.info(f'Local fact found: {self.local_facts[key]}')
            return self.local_facts[key]

        if self.use_wiki:
        # If nothing was found, send request to Wikidata
            self.logger.info(f'Querying Wikidata: {subject} - {relation}')
    
            try:
                sparql = SPARQLWrapper(self.endpoint)
                sparql.setReturnFormat(JSON)
                query = f"""
                SELECT ?object ?objectLabel WHERE {{
                    SERVICE wikibase:mwapi {{
                        bd:serviceParam wikibase:api "EntitySearch";
                            wikibase:endpoint "www.wikidata.org";
                            mwapi:search "{subject}";
                            mwapi:language "en".
                        ?subject wikibase:apiOutputItem mwapi:item.
                    }}
                    SERVICE wikibase:mwapi {{
                        bd:serviceParam wikibase:api "EntitySearch";
                            wikibase:endpoint "www.wikidata.org";
                            mwapi:search "{relation}";
                            mwapi:language "en";
                            mwapi:type "property".
                        ?property wikibase:apiOutputItem mwapi:item.
                    }}
                    ?property wikibase:directClaim ?wdtProperty.
                    ?subject ?wdtProperty ?object.
                    OPTIONAL {{
                        ?object rdfs:label ?objectLabel.
                        FILTER(LANG(?objectLabel) = "en")
                    }}
                }}
                LIMIT 1
                """
                sparql.setQuery(query)
                sparql.setReturnFormat(JSON)
                results = sparql.query().convert()
    
                if results["results"]["bindings"]:
                    result = results["results"]["bindings"][0]
                    obj = result.get("object", {}).get("value", "")
                    label = result.get("objectLabel", {}).get("value", "")
                    self.cache[key] = label
                    self.logger.info(f'Wikidata result: {label}')
                    return label if label else obj.split("/")[-1]
            except Exception as e:
                self.logger.error(f'Wikidata query failed with exception: {str(e)}')
                self.cache[key] = None

        self.logger.info('Nothing found!')
        return None


In [ ]:
class NLIValidator:
    def __init__(self, model_name="roberta-large-mnli", device="cuda:0"):
        self.logger = logging.getLogger('KGEdit.NLIValidator')
        self.logger.info('Initialising NLIValidator...')
        self.device = device
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.contradiction_id = 0
        self.logger.info('NLIValidator initialised!')

    def check_contradiction(self, text1, text2):
        self.logger.info(f'Checking contradiction: "{text1}" vs "{text2}"')
        inputs = self.tokenizer(
            text1, 
            text2, 
            return_tensors="pt",
            truncation=True,
            max_length=512
        ).to(self.device)
        
        with torch.no_grad():
            outputs = self.model(**inputs)
            logits = outputs.logits
            predicted_label = logits.argmax(dim=1).item()

        is_contradiction = predicted_label == self.contradiction_id
        self.logger.info(f'Contradiction check result: {is_contradiction}')
        return is_contradiction


In [ ]:
class KGEditor:
    def __init__(self, model_name: str, use_nli: bool = False, use_chat_template: bool = False, use_wiki: bool = True):
        self.logger = logging.getLogger('KGEdit.KGEditor')
        self.logger.info('Initialising KGEditor...')
        self.model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.tokenizer.pad_token_id = self.tokenizer.eos_token_id
        self.kg = KnowledgeGraph(use_wiki=use_wiki)
        self.nli_validator = NLIValidator(device=str(self.model.device)) if use_nli else None
        self.use_chat_template = use_chat_template
        self.logger.info('KGEditor initialisation complete!')

    def edit_knowledge(self, record):
        self.logger.info(f'Editing knowledge: Subject: {record["subject"]}; Object: {record["target_new"]}; Prompt: {record["prompt"]}')
        if self.use_chat_template:
            messages = [
                {'role': 'system', 'content': 'You are a helpful assistant. Given subject and object, extract relation between them from the given phrase, so that they could form a knowledge triplet.'},
                {'role': 'user', 'content': 'Subject: Albert Einstein\nObject: Ulm\nPhrase: The place of birth of Albert Einstein is\nOutput:'},
                {'role': 'assistant', 'content': 'place of birth'},
                {"role": "user", "content": "Subject: France\nObject: Emmanuel Macron\nPhrase: The current head of state of France is\nOutput:"},
                {'role': 'assistant', 'content': 'head of state'},
                {"role": "user", "content": f'Subject: {record["subject"]}\nObject: {record["target_new"]}\nPhrase: {record["prompt"]}\nOutput:'}
            ]
            text = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )
            inputs = self.tokenizer([text], return_tensors="pt").to(self.model.device)
        else:
            prompt = f"""
            Given subject and object, extract relation between them from the given phrase, so that they could form a knowledge triplet.
            Return ONLY relation as a short phrase, without any additional symbols.
    
            Examples:
            Subject: Albert Einstein
            Object: Ulm
            Phrase: The place of birth of Albert Einstein is
            Output: place of birth
    
            Subject: France
            Object: Emmanuel Macron
            Phrase: The current head of state of France is
            Output: head of state
    
            Now extract from this data:
            Subject: {record["subject"]}
            Object: {record["target_new"]}
            Phrase: {record["prompt"]}
            Output:
            """
            inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        outputs = self.model.generate(
            **inputs, 
            max_new_tokens=50,
            pad_token_id=self.tokenizer.eos_token_id
            # temperature=0.1
        )
        outputs = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, outputs)]
        response = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].strip()
        self.logger.info(f'Model extracted relation from sentence: "{response}"')
        self.kg.add_fact(record["subject"].lower(), response.lower(), record["target_new"].lower())
        self.logger.info('Knowledge added successfully!')

    def process(self, query):
        self.logger.info(f'Processing query: "{query}""')
        entities_relations = self._extract_entities_relations(query)
        facts = self._extract_facts(entities_relations)
        final_answer = self._generate_response(query, facts)
        self.logger.info(f'Generation complete! Final answer: "{final_answer}"')
        return final_answer

    def _extract_entities_relations(self, query):
        self.logger.info(f'Extracting entities/relations from: {query}')      
        if self.use_chat_template:
            messages = [
                {'role': 'system', 'content': 'You are a helpful assistant. You task is to Extract ONLY the main entities and relations from the query. Return EXACTLY ONE line in the format: [ENTITY|RELATION]'},
                {"role": "user", "content": f'Where was Albert Einstein born?'},
                {'role': 'assistant', 'content': '[Albert Einstein|place of birth]'},
                {"role": "user", "content": f'Who is the current president of France?'},
                {'role': 'assistant', 'content': '[France|head of state]'},
                {"role": "user", "content": query}
            ]
            text = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )
            inputs = self.tokenizer([text], return_tensors="pt").to(self.model.device)
        else:
            text = f"""
            Extract ONLY the main entities and relations from the query.
            Return EXACTLY ONE line in the format: [ENTITY|RELATION]
    
            Examples:
            Query: Where was Albert Einstein born?
            Output: [Albert Einstein|place of birth]
    
            Query: Who is the current president of France?
            Output: [France|head of state]
    
            Query: What is the capital of Germany?
            Output: [Germany|capital]
    
            Now extract from this query:
            Query: {query}
            Output:
            """
            inputs = self.tokenizer(text, return_tensors="pt").to(self.model.device)
        outputs = self.model.generate(
            **inputs, 
            max_new_tokens=30,
            pad_token_id=self.tokenizer.eos_token_id
            # temperature=0.1
        )
        outputs = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, outputs)]
        response = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].strip()
        if "|" in response:
            if "[" not in response:
                response = '[' + response
            if ']' not in response:
                response = response + ']'

        self.logger.info(f"Model extraction response: {response}")

        entities_relations = []
        if "[" in response and "|" in response and "]" in response:
            try:
                content = response.split("[")[1].split("]")[0]
                parts = content.split("|", 1)
                if len(parts) == 2:
                    entities_relations.append((parts[0].strip(), parts[1].strip()))
            except Exception as e:
                self.logger.error(f"Failed to parse extraction response. Error message: {e}")

        # Fallback if no entities found
        if not entities_relations:
            self.logger.warning("No entities/relations found, using fallback")
            return [(query, "unknown")]
        return entities_relations

    def _extract_facts(self, entities_relations):
        self.logger.info(f'Extracting facts from KG for {len(entities_relations)} entities/relations')
        facts = []
        for subject, relation in entities_relations:
            fact = self.kg.query(subject, relation)
            if fact:
                facts.append(f"{subject}|{relation}|{fact}")
        self.logger.info(f'Extracted {len(facts)} facts')
        return facts

    def _generate_response(self, query, facts):
        self.logger.info(f'Generating response for query: "{query}"')
        if self.use_chat_template:
            if facts:
                messages = [
                    {'role': 'system', 'content': 'You are taking part in scientific experiment. Please, answer a given question with respect to given facts. If no facts provided or provided facts do not suffice, use your own knowledge.'},
                    {"role": "user", "content": f"Question: {query}\nFacts: {facts}"}
                ]
            else:
                messages = [
                    {'role': 'system', 'content': 'You are taking part in scientific experiment. Please, answer any given question.'},
                    {"role": "user", "content": query}
                ]
            text = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )
            inputs = self.tokenizer([text], return_tensors="pt").to(self.model.device)
        else:
            if facts:
                text = f"""Please, answer a given question with respect to given facts. If provided facts do not suffice, use your own knowledge.
                Question: {query}
                Facts: {facts}
                """
            else:
                text = f"""Please, answer a given question.
                Question: {query}
                """
            inputs = self.tokenizer(text, return_tensors="pt").to(self.model.device)
            
        outputs = self.model.generate(
            **inputs, 
            max_new_tokens=50,
            pad_token_id=self.tokenizer.eos_token_id
            # temperature=0.1
        )
        outputs = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, outputs)]
        response = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].strip()
        self.logger.info(f'Initial response: "{response}"')

        if facts and not self._check_response(response, facts):
            self.logger.info("Response needs correction based on facts")
        
            correction_prompt = (
                f"Original query: {query}\n"\
                f"Initial response: {response}\n"\
                f"Verified facts: {', '.join(facts)}\n"\
                "Please generate a corrected response:"
            )
            return self._generate_response(correction_prompt, [])
        return response
            
    def _check_response(self, response, facts):
        self.logger.info(f'Checking response for correctness. Response: "{response}". Facts: "{facts}"')
        if not facts:
            return True

        if self.nli_validator:
            self.logger.info("Using NLI validator to check correctness")
            for fact in facts:
                if self.nli_validator.check_contradiction(response, fact):
                    return False
            return True
        else:
            # Simple keyword check
            return any(fact.lower().split('|')[-1] in response.lower() for fact in facts)


In [ ]:
def flatten_ground_truth(ground_truth):
    flat_list = []
    stack = [ground_truth]
    
    while stack:
        current = stack.pop()
        if isinstance(current, list):
            stack.extend(reversed(current))
        else:
            flat_list.append(str(current).lower())
    
    return flat_list


In [ ]:
def eval_kgedit(result_path: str, logger):
    logger.info(f'Evaluating results from: {result_path}')
    if not os.path.exists(result_path):
        raise FileNotFoundError(f'File {result_path} not found!')
    
    with open(result_path, 'r') as f:
        datas = json.load(f)
    
    # Edit Success (Rewrite Accuracy)
    edit_succ_list = []
    for data in datas:
        if 'rewrite_acc' in data['post']:
            edit_succ_list.extend(data['post']['rewrite_acc'])
    edit_succ = sum(edit_succ_list) / len(edit_succ_list) * 100 if edit_succ_list else 0
    logger.info(f'Edit Success: {edit_succ:.2f}%')
    
    # Portability
    portability_scores = []
    portability_metrics = {}
    for data in datas:
        if 'portability' in data['post']:
            for key, values in data['post']['portability'].items():
                portability_scores.extend(values)
                if key not in portability_metrics:
                    portability_metrics[key] = []
                portability_metrics[key].extend(values)
    
    if portability_scores:
        overall_portability = sum(portability_scores) / len(portability_scores) * 100
        logger.info(f'Overall Portability: {overall_portability:.2f}%')
        for key, values in portability_metrics.items():
            avg = sum(values) / len(values) * 100 if values else 0
            logger.info(f'  - {key}: {avg:.2f}%')
    
    # Locality
    locality_scores = []
    locality_metrics = {}
    for data in datas:
        if 'locality' in data['post']:
            for key, values in data['post']['locality'].items():
                locality_scores.extend(values)
                if key not in locality_metrics:
                    locality_metrics[key] = []
                locality_metrics[key].extend(values)
    
    if locality_scores:
        overall_locality = sum(locality_scores) / len(locality_scores) * 100
        logger.info(f'Overall Locality: {overall_locality:.2f}%')
        for key, values in locality_metrics.items():
            avg = sum(values) / len(values) * 100 if values else 0
            logger.info(f'  - {key}: {avg:.2f}%')
    
    print("\n===== Final Evaluation Results =====")
    print(f"Edit Success: {edit_succ:.2f}%")
    
    if portability_scores:
        print(f"\nOverall Portability: {overall_portability:.2f}%")
        for key, values in portability_metrics.items():
            avg = sum(values) / len(values) * 100
            print(f"  - {key}: {avg:.2f}%")
    
    if locality_scores:
        print(f"\nOverall Locality: {overall_locality:.2f}%")
        for key, values in locality_metrics.items():
            avg = sum(values) / len(values) * 100
            print(f"  - {key}: {avg:.2f}%")
    
    print("===================================")


In [ ]:
def run_kgedit(data_path, data_size, model_name, use_nli=False, use_chat_template=False, use_wiki=True, out_path='kgedit_results.json'):
    logger = logging.getLogger('KGEdit.run')
    logger.info('Starting KGEdit pipeline')
    logger.info(f'Configuration: model={model_name}, use_nli={use_nli}, use_chat_template={use_chat_template}')
    
    kgeditor = KGEditor(model_name, use_nli, use_chat_template, use_wiki)
    
    dataset = KnowEditDataset(data_path, size=data_size)
    logger.info(f'Loaded dataset with {len(dataset)} records')
    
    if 'counterfact' in data_path:
        datatype = 'counterfact'
    elif 'recent' in data_path:
        datatype = 'recent'
    elif 'wikibio' in data_path:
        datatype = 'wikibio'
    elif 'ZsRE' in data_path:
        datatype = 'zsre'
    else:
        logger.warning('Unknown dataset type, assuming counterfact format')
        datatype = 'counterfact'

    logger.info(f'Detected dataset type: {datatype}')

    logger.info('Applying knowledge edits')
    for record in tqdm(dataset, desc='Applying knowledge edits'):
        # logger.info(f'RECORD: {record}')
        kgeditor.edit_knowledge(record)

    logger.info('Evaluating method')
    results = []
    for i, record in enumerate(tqdm(dataset, desc=f'Evaluating')):
        metrics = {
            'case_id': i,
            'requested_rewrite': record,
            'post': {}
        }
        
        # 1. rewrite accuracy
        response = kgeditor.process(record["prompt"])
        target_new = record["target_new"].strip().lower()
        rewrite_acc = 1.0 if target_new in response.lower() else 0.0
        metrics["post"]["rewrite_acc"] = [rewrite_acc]

        # 2. Portability
        metrics["post"]["portability"] = {}
        if datatype in ['counterfact', 'recent', 'zsre']:
            portability_keys = {
                'Subject_Aliasing': 'portability_s',
                'reasoning': 'portability_r',
                'Logical_Generalization': 'portability_l'
            }
            for metric_name, record_key in portability_keys.items():
                if record_key in record and record[record_key] is not None:
                    acc_list = []
                    for test in record[record_key]:
                        if test is None:
                            continue
                        test_response = kgeditor.process(test["prompt"]).lower()
                        ground_truths = flatten_ground_truth(test["ground_truth"])
                        correct = any(truth in test_response for truth in ground_truths)
                        acc_list.append(1.0 if correct else 0.0)
                    metrics["post"]["portability"][metric_name] = acc_list

        # 3. Locality
        metrics["post"]["locality"] = {}
        if datatype in ['counterfact', 'recent', 'zsre']:
            locality_keys = {
                'Relation_Specificity': 'locality_rs',
                'Forgetfulness': 'locality_f'
            }
            for metric_name, record_key in locality_keys.items():
                if record_key in record and record[record_key] is not None:
                    acc_list = []
                    for test in record[record_key]:
                        if test is None:
                            continue
                        test_response = kgeditor.process(test["prompt"]).lower()
                        ground_truths = flatten_ground_truth(test["ground_truth"])
                        correct = any(truth in test_response for truth in ground_truths)
                        acc_list.append(1.0 if correct else 0.0)
                    metrics["post"]["locality"][metric_name] = acc_list
        elif datatype == 'wikibio':
            if 'locality_rs' in record and record['locality_rs'] is not None:
                acc_list = []
                for test in record['locality_rs']:
                    if test is None:
                        continue
                    test_response = kgeditor.process(test["prompt"]).lower()
                    ground_truths = flatten_ground_truth(test["ground_truth"])
                    correct = any(truth in test_response for truth in ground_truths)
                    acc_list.append(1.0 if correct else 0.0)
                metrics["post"]["locality"]['Relation_Specificity'] = acc_list

        results.append(metrics)
        
    logger.info('Evaluating completed. Saving results.')
    with open(out_path, 'w') as f:
        json.dump(results, f, indent=4)

    eval_kgedit(out_path, logger)
    logger.info('KGEdit pipeline completed successfully')


In [ ]:
SAMPLE = 'recent'
SAMPLE_TYPE = 'train'

train_data_dirs = {
    'wikibio': '/kaggle/working/EasyEdit/data/benchmark/WikiBio/wikibio-train-all.json', # OK
    'counterfact': '/kaggle/working/EasyEdit/data/benchmark/wiki_counterfact/train_cf.json', # OK
    'recent': '/kaggle/working/EasyEdit/data/benchmark/wiki_recent/recent_train.json', # OK
    'zsre': None
}
test_data_dirs = {
    'wikibio': '/kaggle/working/EasyEdit/data/benchmark/WikiBio/wikibio-test-all.json', # OK
    'counterfact': '/kaggle/working/EasyEdit/data/benchmark/wiki_counterfact/test_cf.json', # OK
    'recent': '/kaggle/working/EasyEdit/data/benchmark/wiki_recent/recent_test.json', # OK
    'zsre': '/kaggle/working/EasyEdit/data/benchmark/ZsRE/ZsRE-test-all.json'
}

sample_path = test_data_dirs[SAMPLE] if SAMPLE_TYPE == 'test' else train_data_dirs[SAMPLE]

qwen_models = {
    'qwen2.5-1.5b': '/kaggle/input/qwen2.5/transformers/1.5b/1',
    'qwen2.5-1.5b-instruct': '/kaggle/input/qwen2.5/transformers/1.5b-instruct/1',
    'qwen2.5-3b': '/kaggle/input/qwen2.5/transformers/3b/1',
    'qwen2.5-3b-instruct': '/kaggle/input/qwen2.5/transformers/3b-instruct/1',
    'gpt2-xl': 'openai-community/gpt2-xl'
}


run_kgedit(
    data_path=sample_path,
    data_size=50,
    model_name=qwen_models[MODEL],
    use_nli=True,
    use_chat_template=USE_CHAT_TEMPLATE,
    use_wiki=True
)


config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Applying knowledge edits:   0%|          | 0/50 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]


===== Final Evaluation Results =====
Edit Success: 72.00%

Overall Portability: 35.29%
  - reasoning: 27.27%
  - Logical_Generalization: 60.00%
  - Subject_Aliasing: 0.00%

Overall Locality: 82.78%
  - Relation_Specificity: 81.22%
  - Forgetfulness: 92.86%


# Baselines

In [ ]:
!git clone https://github.com/zjunlp/EasyEdit
%cd EasyEdit
!git checkout f0ec5eb3ef6a09f637fa5344667a78f38fbeb05d
!pip install -r colab_requirements.txt
!pip install qwen-vl-utils
!pip install av
!pip install openai==1.2.0
!pip install zhipuai
!pip install hydra-core -U
!pip install sentence_transformers==3.2.1
!pip install -U datasets
!pip install -q sparqlwrapper


Cloning into 'EasyEdit'...
remote: Enumerating objects: 8523, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 8523 (delta 7), reused 3 (delta 3), pack-reused 8502 (from 2)
Receiving objects: 100% (8523/8523), 85.85 MiB | 42.92 MiB/s, done.
Resolving deltas: 100% (5411/5411), done.
/kaggle/working/EasyEdit
Note: switching to 'f0ec5eb3ef6a09f637fa5344667a78f38fbeb05d'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at f0ec5eb fix bug in notebooks an

In [ ]:
from datasets import load_dataset
from easyeditor import KnowEditDataset, ZsreDataset, EditTrainer # , BaseEditor
from easyeditor import (
    FTHyperParams, 
    IKEHyperParams, 
    KNHyperParams, 
    MEMITHyperParams, 
    ROMEHyperParams, 
    LoRAHyperParams,
    MELOHyperParams,
    MENDHyperParams,
    MENDTrainingHparams,
    SERACHparams
    )
import re
import json
import os
import os.path as path
import numpy as np


In [ ]:
USE_CHAT_TEMPLATE = False
MODEL = 'qwen2.5-1.5b-instruct' # gpt2-xl, qwen2.5-1.5b
if 'instruct' in MODEL:
    USE_CHAT_TEMPLATE = True


In [ ]:
# generate_fast

import unicodedata
from typing import List, Optional

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from easyeditor.util.logit_lens import LogitLens

def generate_fast(
    model: AutoModelForCausalLM,
    tok: AutoTokenizer,
    prompts: List[str],
    n_gen_per_prompt: int = 1,
    top_k: int = 5,
    max_out_len: int = 200,
    vanilla_generation=False
):
    """
    Changed function with added ability to use chat template generation
    """

    if USE_CHAT_TEMPLATE and hasattr(tok, "apply_chat_template"):
        # print('applying chat template!')
        formatted_prompts = []
        for prompt in prompts:
            messages = [{"role": "user", "content": prompt}]
            try:
                formatted_prompt = tok.apply_chat_template(
                    messages,
                    tokenize=False,
                    add_generation_prompt=True
                )
                formatted_prompts.append(formatted_prompt)
            except Exception as e:
                print(f"Error applying chat template: {e}")
                formatted_prompts.append(prompt)
        prompts = formatted_prompts

    # Unroll prompts and tokenize
    inp = [prompt for prompt in prompts for _ in range(n_gen_per_prompt)]
    inp_tok = tok(inp, padding=True, return_tensors="pt").to(
        next(model.parameters()).device
    )
    input_ids, attention_mask = inp_tok["input_ids"], inp_tok["attention_mask"]
    if vanilla_generation:
        gen_txt = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=max_out_len
        )
        txt = [tok.decode(x, skip_special_tokens=True) for x in gen_txt.detach().cpu().numpy().tolist()]
        txt = [
            unicodedata.normalize("NFKD", x)
            .replace("\n\n", " ")
            .replace("<|endoftext|>", "")
            for x in txt
        ]
        return txt
    batch_size = input_ids.size(0)

    # Setup storage of fast generation with attention caches.
    # `cur_context` is used to define the range of inputs that are not yet
    # stored in `past_key_values`. At each step, we are generating the
    # next token for the index at `cur_context.stop + 1`.
    past_key_values, cur_context = None, slice(0, attention_mask.sum(1).min().item())

    with torch.no_grad():
        while input_ids.size(1) < max_out_len:  # while not exceeding max output length
            model_out = model(
                input_ids=input_ids[:, cur_context],
                attention_mask=None if 'llama' in model.name_or_path.lower() or 'baichuan' in model.name_or_path.lower() or 'internlm' in model.name_or_path.lower() else attention_mask[:, cur_context],
                past_key_values=past_key_values,
                use_cache=True,
            )
            if type(model_out) is torch.Tensor:
                logits = model_out
            else:
                logits = model_out.logits
            past_key_values = model_out.past_key_values
            softmax_out = torch.nn.functional.softmax(logits[:, -1, :], dim=1)

            # Top-k sampling
            tk = torch.topk(softmax_out, top_k, dim=1).indices
            softmax_out_top_k = torch.gather(softmax_out, 1, tk)
            softmax_out_top_k = softmax_out_top_k / softmax_out_top_k.sum(1)[:, None]
            new_tok_indices = torch.multinomial(softmax_out_top_k, 1)
            new_toks = torch.gather(tk, 1, new_tok_indices)

            # If we're currently generating the continuation for the last token in `input_ids`,
            # create a new index so we can insert the new token
            if cur_context.stop == input_ids.size(1):
                attention_mask = torch.cat(
                    [attention_mask, attention_mask.new_zeros(batch_size, 1)], dim=1
                )
                input_ids = torch.cat(
                    [
                        input_ids,
                        input_ids.new_ones(batch_size, 1) * tok.pad_token_id,
                    ],
                    dim=1,
                )

            last_non_masked = attention_mask.sum(1) - 1
            for i in range(batch_size):
                new_idx = last_non_masked[i] + 1
                if last_non_masked[i].item() + 1 != cur_context.stop:
                    continue

                # Stop generating if we've already maxed out for this prompt
                if new_idx < max_out_len:
                    input_ids[i][new_idx] = new_toks[i]
                    attention_mask[i][new_idx] = 1

            cur_context = slice(cur_context.stop, cur_context.stop + 1)
    txt = [tok.decode(x, skip_special_tokens=True) for x in input_ids.detach().cpu().numpy().tolist()]
    txt = [
        unicodedata.normalize("NFKD", x)
        .replace("\n\n", " ")
        .replace("<|endoftext|>", "")
        for x in txt
    ]

    return txt


In [ ]:
# evaluate_utils.py

import torch
import numpy as np
import scipy
import nltk
import typing
# from ..util.generate import generate_fast
import torch.nn.functional as F
from easyeditor.trainer import *
from sklearn.metrics import f1_score
import openai
from openai import OpenAI
from transformers import T5ForConditionalGeneration
import time
import regex
import string


def normalize_answer(s):
    def remove_articles(text):
        return regex.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def exact_match_score(prediction, ground_truth):
    return normalize_answer(prediction) == normalize_answer(ground_truth)

def llm_judge(question, ground_truth, prediction, api_key):
    content_template = """
Your job is to look at a question, a gold target, and a predicted answer, and then assign a grade of either ["CORRECT", "INCORRECT"].

The following are examples of CORRECT predicted answers.
```
Question: What are the names of Barack Obama's children?
Gold target: Malia Obama and Sasha Obama
Predicted answer 1: sasha and malia obama
Predicted answer 2: Malia and Sasha Obama are the names of Barack Obama's children.
```
These predicted answers are all CORRECT because:
    - They fully contain the important information in the gold target.
    - They do not contain any information that contradicts the gold target.

The following are examples of INCORRECT predicted answers.
```
Question: What are the names of Barack Obama's children?
Gold target: Malia and Sasha
Predicted answer 1: Malia.
Predicted answer 2: Malia, Sasha, and Susan.
Predicted answer 3: Malia and Sasha, Malia and Sasha, Malia and Sasha, Malia and Sasha (repeated answer)
```
These predicted answers are all INCORRECT because:
    - A factual statement in the answer contradicts the gold target or contain repeated answer.


Here is a sample. Simply reply with either CORRECT or INCORRECT.

```
Question: {question}
Gold target: {target}
Predicted answer: {predicted_answer}
```

According to the gold target, please grade the predicted answer of this question as one of:
A: CORRECT
B: INCORRECT

Just return the letters "A" or "B", with no text around it.
    """.strip()

    content = content_template.format(
        question=question,
        target=ground_truth,
        predicted_answer=prediction,
    )

    client = OpenAI(
        api_key=api_key,
        base_url="https://api.deepseek.com"
    )

    completion = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": ""},
            {"role": "user", "content": content}
        ],
        temperature=0.0
    )
    llm_ans = completion.choices[0].message.content
    llm_score = 1.0 if llm_ans == "A" else 0.0
    time.sleep(1) # avoid high rate of request
    return llm_score

def test_prediction_acc_LLM_judge(model, tok, hparams, prompts, targets, device, locality=False):
    # generation & truncation
    all_score = []
    all_response = []
    if isinstance(prompts, str):
        prompts, targets = [prompts, ], [targets, ]
    for prompt in prompts:
        messages = [
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
        text = tok.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True,
            )
        prompt_tok = tok(
            text,
            padding = True,
            truncation = True,
            return_tensors="pt",
        ).to(f"cuda:{device}")
        # add a template
        gen_tokens = model.generate(
            input_ids=prompt_tok['input_ids'],
            attention_mask=prompt_tok['attention_mask'],
            max_new_tokens=512,
            stop_strings=[".", "\n", tok.eos_token],
            tokenizer=tok,
            pad_token_id=tok.eos_token_id,
            do_sample=False,
            use_cache=False,
        )
        # decode and process
        if isinstance(model, T5ForConditionalGeneration):
            trunc_gen_tokens = gen_tokens[0]  # encoder-decoder model only provied generated content after prompt
        else:
            trunc_gen_tokens = gen_tokens[0][prompt_tok['input_ids'].shape[1]:]  # decoder-only model provied generated content containing prompt
        # if locality:
        #     ans = trunc_gen_tokens.detach().cpu().numpy().tolist()
        #     all_response.append(ans)
        # else:
        gen_content = tok.decode(trunc_gen_tokens)
        suffixes_to_remove = [".", "\n", tok.eos_token]
        for suffix in suffixes_to_remove:
            if gen_content.endswith(suffix):
                gen_content = gen_content[:-len(suffix)]
        # LLM-as-a-Judge
        if hparams.evaluation_type == "generate-text":
            all_response.append(gen_content)
        elif hparams.evaluation_type == "LLM-judge" and hasattr(hparams, 'api_key') and hparams.api_key:
            LLM_Score = llm_judge(prompts, targets, gen_content, hparams.api_key)
            all_score.append(LLM_Score)
            all_response.append(gen_content)
        else:
            # the user do not provide api key, using exact match as an alternative
            EM_Score = float(exact_match_score(gen_content, targets))
            all_score.append(EM_Score)
            all_response.append(gen_content)
    
    if len(all_score) > 0:
        return all_score, all_response
    else:
        return all_response

def test_batch_prediction_acc(model, tok, hparams, prompts, target, device, locality=False):
    prompt_tok = tok(
        prompts,
        padding=True,
        truncation=True,
        max_length=hparams.max_length,
        return_tensors="pt",
    ).to(f"cuda:{device}")

    with torch.no_grad():
        outputs = model(**prompt_tok)
        if type(outputs) is torch.Tensor:
            logits = outputs
        else:
            logits = outputs.logits

        if tok.padding_side == 'left':
            ans = torch.argmax(logits, dim=-1)[:, -1].squeeze()
        else:
            last_non_masked = prompt_tok["attention_mask"].sum(1) - 1
            to_gather = last_non_masked.unsqueeze(1).repeat(1, logits.size(-1)).unsqueeze(1)
            gathered = torch.gather(logits, 1, to_gather).squeeze(1)
            ans = torch.argmax(gathered, dim=1)

        ans = ans.squeeze().detach().cpu().numpy().tolist()

        if locality:
            return ans

        return np.mean(np.equal(ans, target))

def test_seq2seq_batch_prediction_acc(model, tok, hparams, prompts, targets, device, locality=False):
    if isinstance(prompts, str):
        prompts,targets = [prompts,], [targets,]
    prompt_tok = tok(
        prompts,
        padding=True,
        truncation=True,
        max_length=hparams.max_length,
        return_tensors="pt",
    ).to(f"cuda:{device}")

    trg_tok = tok(
        targets,
        padding=True,
        truncation=True,
        max_length=hparams.max_length,
        return_tensors="pt",
    ).to(f"cuda:{device}")

    prompt_tok['decoder_input_ids'] = trg_tok['input_ids']
    prompt_tok['decoder_attention_mask'] = trg_tok['attention_mask']

    with torch.no_grad():
        outputs = model(**prompt_tok)
        if type(outputs) is torch.Tensor:
            logits = outputs
        else:
            logits = outputs.logits

        assert logits.size(1) == trg_tok['input_ids'].size(1)
        ans = torch.argmax(logits, dim=-1)
        if locality:
            answers = ans.squeeze().detach().cpu().numpy().tolist()
            return answers if type(answers[0]) is list else [answers,]
        return torch.mean((trg_tok['input_ids'][:,:-1] == ans[:,:-1]).float(), dim=-1).detach().cpu().numpy().tolist()

def test_prediction_acc(model, tok, hparams, prompts, targets, device, locality=False, vanilla_generation=False):
    if vanilla_generation:
        if isinstance(prompts, str):
            prompts, targets = [prompts, ], [targets, ]
        results = []
        for prompt, target_new in zip(prompts, targets):
            target_new_tokens = tok.encode(target_new, add_special_tokens=False)
            prompt_tok = tok(
                prompt,
                return_tensors="pt",
            ).to(f"cuda:{device}")
            gen_token = model.generate(
                input_ids=prompt_tok['input_ids'],
                attention_mask=prompt_tok['attention_mask'],
                max_new_tokens=len(target_new_tokens),
                pad_token_id=tok.eos_token_id,
                do_sample=False,
                use_cache=False,
            )
            if locality:
                results.append(gen_token.detach().cpu().numpy().tolist()[0][-len(target_new_tokens):])
            else:
                results.append(np.mean(np.equal(target_new_tokens, gen_token.detach().cpu().numpy().tolist()[0][-len(target_new_tokens):])))
        return results

    if isinstance(prompts, str):
        prompts,targets = [prompts,], [targets,]
    if not locality and hasattr(hparams, 'use_chat_template') and hparams.use_chat_template:
        prompts = [[{"role":"user", "content":m}] for m in prompts]
        prompts=tok.apply_chat_template(prompts,
                                        add_generation_prompt=True,
                                        tokenize=False)
    prompt_target = [prompt + ' ' + target for prompt, target in zip(prompts,targets)]
    max_prompt_len = max([len(tok.encode(_)) for _ in prompt_target]) + 1
    before_padding_side = tok.padding_side
    tok.padding_side = 'left'
    prompt_target_tok = tok(
        prompt_target,
        padding=True,
        truncation=True,
        max_length=max(hparams.max_length, max_prompt_len),
        return_tensors="pt",
    ).to(f"cuda:{device}")
    prompt_tok = tok(
        prompts,
        padding=True,
        truncation=True,
        max_length=max(hparams.max_length, max_prompt_len),
        return_tensors="pt",
    )
    tok.padding_side = before_padding_side
    num_prompt_toks = [int((i != tok.pad_token_id).sum()) for i in prompt_tok['input_ids']]
    num_pad_toks = [int((i == tok.pad_token_id).sum()) for i in prompt_target_tok['input_ids'].cpu()]
    prompt_len = [x+y for x,y in zip(num_pad_toks,num_prompt_toks)]
    with torch.no_grad():
        outputs = model(**prompt_target_tok)
        if type(outputs) is torch.Tensor:
            logits = outputs
        else:
            logits = outputs.logits
        answers = torch.argmax(logits, dim=-1).squeeze().detach().cpu().numpy().tolist()
        labels = prompt_target_tok['input_ids'].squeeze().detach().cpu().numpy().tolist()
        answers = slice_list(answers,prompt_len,left=True)
        labels = slice_list(labels,prompt_len,left=False)
        if locality:
            return answers if type(answers[0]) is list else [answers,]
        if isinstance(answers[0], list):
            res = []
            for ans,label in zip(answers,labels):
                temp_acc = np.mean(np.equal(ans, label))
                if np.isnan(temp_acc):
                    continue
                res.append(temp_acc)
            return res
        else:
            return [np.mean(np.equal(answers, labels))]

def test_generation_quality_serac(
    model,
    tok,
    prefixes: typing.List[str],
    max_out_len: int,       
):
    #only single case
    prompt_tok = tok(
        prefixes,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )
    prompt_tok_length=len(prompt_tok['input_ids'])
    gen_texts=model.generate(**prompt_tok,max_new_tokens=256)
    if isinstance(model,SERAC):
        gen_texts=tok.decode(gen_texts[prompt_tok_length:])
        gen_texts=[gen_texts]
        print(len(gen_texts))
    else:
        gen_texts=tok.decode(gen_texts[prompt_tok_length:])
        gen_texts=[gen_texts]
        print(len(gen_texts))      
    ngram_entropy = n_gram_entropy(gen_texts, return_list=True)


    ret = {
        "ngram_entropy": ngram_entropy
    }
    return ret

def test_generation_quality(
    model,
    tok,
    prefixes: typing.List[str],
    max_out_len: int,
    vanilla_generation: bool = False,
):
    gen_texts = generate_fast(
        model,
        tok,
        prefixes,
        n_gen_per_prompt=1,
        max_out_len=max_out_len,
        vanilla_generation=vanilla_generation,
    )

    ngram_entropy = n_gram_entropy(gen_texts)
    ret = {
        "ngram_entropy": ngram_entropy,
    }
    return ret

def n_gram_entropy(gen_texts, agg="arith"):
    assert agg in ["arith", "geom"]

    return (scipy.stats.mstats.gmean if agg == "geom" else np.mean)(
        [compute_n_gram_entropy(txt) for txt in gen_texts]
    ).item()

def compute_n_gram_entropy(sentence, ns=None, weights=None, agg="arith"):
    if ns is None:
        ns = [2, 3]
    if weights is None:
        weights = [2 / 3, 4 / 3]
    assert agg in ["arith", "geom"]

    entropy_list = []
    for n in ns:
        fdist = compute_freq(sentence, n)
        freqs = np.array([freq for _, freq in fdist.items()])
        freqs = freqs / freqs.sum()

        entropy_list.append(np.sum(-freqs * np.log(freqs) / np.log(2)))

    entropy_list = np.array(entropy_list) * np.array(weights)

    return (scipy.stats.mstats.gmean if agg == "geom" else np.mean)(entropy_list)

def compute_freq(sentence, n=2):
    tokens = nltk.word_tokenize(sentence)
    ngrams = nltk.ngrams(tokens, n)
    return nltk.FreqDist(ngrams)

def PPL(
    model,
    tok,
    prompt: typing.Union[str, typing.List[str]],
    target_new: typing.Union[str, typing.List[str]],
    device,
):
    if isinstance(prompt, str):
        prompt,target_new = [prompt,], [target_new,]
    full_prompt = [f"{p} {l}" for p, l in zip(prompt, target_new)]
    prompt_ids = tok(list(prompt), return_tensors="pt", padding=True, truncation=True)["input_ids"]
    num_prompt_toks = [int((i != tok.pad_token_id).sum()) for i in prompt_ids]
    tokens = tok(full_prompt, return_tensors="pt", padding=True, truncation=True)
    tokens["labels"] = tokens["input_ids"].clone()
    for i in range(len(prompt)):
        tokens["labels"][i][:num_prompt_toks[i]] = -100
    tokens["labels"][tokens["input_ids"] == tok.pad_token_id] = -100 # What is this doing?
    batch = {f"{k1}" : v1 for k1, v1 in tokens.items()}
    input_ids = batch["input_ids"][:, :1024]#.to(device)
    if "labels" not in batch:
        target_ids = batch["input_ids"][:, :1024].clone()
    else:
        target_ids = batch["labels"][:, :1024].clone()
    with torch.no_grad():
        outputs = model(input_ids=input_ids.to(device), labels=target_ids.to(device))
        nll = outputs.loss
    ppl = torch.exp(nll)#.clip(0, 100)
    return ppl.cpu().numpy().tolist()


def OOD_PPL(
        model,
        tok,
        prompt: typing.Union[str, typing.List[str]],
        target_new: typing.Union[str, typing.List[str]],
        device,
        threshold=0.8
):
    if isinstance(prompt, str):
        prompt, target_new = [prompt, ], [target_new, ]

    full_prompt = [f"{p}" for p, l in zip(prompt, target_new)]
    tokens = tok(full_prompt, return_tensors="pt", padding=True, truncation=True)

    tokens["labels"] = tokens['input_ids'].clone()
    tokens["labels"][tokens["input_ids"] == tok.pad_token_id] = -100
    batch = {f"{k1}": v1 for k1, v1 in tokens.items()}
    input_ids = batch["input_ids"][:, :1024]  # .to(device)
    target_ids = batch["labels"][:, :1024]

    with torch.no_grad():
        logits = model(input_ids=input_ids.to(device), labels=target_ids.to(device)).logits
        shift_logits = logits[:, :-1, :].contiguous()
        shift_labels = target_ids.to(device)[:, 1:].contiguous()

        log_probs = -nn.functional.log_softmax(shift_logits, dim=-1)
        if shift_labels.dim() == log_probs.dim() - 1:
            shift_labels = shift_labels.unsqueeze(-1)

        padding_mask = shift_labels.eq(-100)

        # In case the ignore_index is -100, the gather will fail, so we replace labels by 0. The padding_mask
        # will ignore them in any case.
        shift_labels = torch.clamp(shift_labels, min=0)

        nll_loss = log_probs.gather(dim=-1, index=shift_labels)
        nll_loss.masked_fill_(padding_mask, 0.0)

        threshold = -np.log(threshold)

        return len(nll_loss[nll_loss < threshold]) / len(nll_loss.view(-1))

def verify_answer(model_answer, correct_answer):
    if type(correct_answer) is str:
        correct_answer = [[correct_answer]]
    for answer in correct_answer:
        if True not in [possible_answer in model_answer for possible_answer in answer]:
            return False
    return True

def answer_match(
    model,
    tok,
    prompt: str,
    target_new: str,
    device,
):
    inputs = tok.encode(prompt, return_tensors='pt').to(device)
    outputs = model.generate(inputs, temperature=0, max_new_tokens=30)
    predict = tok.decode(outputs[0], skip_special_tokens=True)

    return verify_answer(predict,target_new)

def slice_list(matrix,start_indices,left):
    if isinstance(matrix[0], list):
        if left:
            return [row[start_index-1:-1] for row, start_index in zip(matrix, start_indices)]
        else:
            return [row[start_index:] for row, start_index in zip(matrix, start_indices)]
    else:
        if left:
            return matrix[start_indices[0]-1:-1]
        else:
            return matrix[start_indices[0]:]

def gather_log_probs(logits, labels):
    # print(f"labels.shape: {labels.shape} , logits.shape[:-1] :{logits.shape[:-1]}")
    assert labels.dim() == logits.dim() - 1
    assert labels.shape == logits.shape[:-1]
    return logits.log_softmax(-1).gather(-1, labels.unsqueeze(-1)).squeeze(-1)

def masked_mean(values, mask):
    assert mask.dtype == torch.bool
    assert values.shape == mask.shape
    return (values * mask.float()).sum() / mask.sum().float()

def mask_hf_labels(labels, null_token=0):
    valid_mask = labels != -100
    valid_labels = labels.masked_fill(~valid_mask, null_token)
    return valid_mask, valid_labels

def es(pre_logits, edit_logits, q_mask, labels, same_mask):
    
    _, targ = mask_hf_labels(labels)

    pos_mask = same_mask.unsqueeze(-1) * q_mask 
    neg_mask = (~same_mask).unsqueeze(-1) * q_mask 
        
    pre_token_log_probs = gather_log_probs(pre_logits, targ)
    edit_token_log_probs = gather_log_probs(edit_logits, targ)

    mean_pos_pre = masked_mean(pre_token_log_probs, pos_mask)
    mean_pos_edit = masked_mean(edit_token_log_probs, pos_mask)
    mean_neg_edit = masked_mean(edit_token_log_probs, neg_mask)

    z_sent = (mean_pos_edit - mean_neg_edit).sigmoid()
    z_topic_raw = (mean_pos_edit - mean_pos_pre).exp()
    z_topic = min(1, z_topic_raw)

    es_sent = z_sent * z_topic
    return es_sent

def es_per_icl(example, pre_logits, edit_logits):
    with torch.no_grad():
        
        pre_q_mask = example["outer_pre"]["q_mask"]
        edit_q_mask = example["outer_edit"]["q_mask"]
        
        pre_labels = example["outer_pre"]["labels"]
        edit_labels = example["outer_edit"]["labels"]
        
        pre_mask, pre_targ = mask_hf_labels(pre_labels)
        edit_mask, edit_targ = mask_hf_labels(edit_labels)
        
        same_per_mask = example["same_per_mask"]

        pre_pos_mask = same_per_mask.unsqueeze(-1) * pre_q_mask 
        pre_neg_mask = (~same_per_mask).unsqueeze(-1) * pre_q_mask 
        edit_pos_mask = same_per_mask.unsqueeze(-1) * edit_q_mask 
        edit_neg_mask = (~same_per_mask).unsqueeze(-1) * edit_q_mask 
        
        pre_token_log_probs = gather_log_probs(pre_logits, pre_targ)
        edit_token_log_probs = gather_log_probs(edit_logits, edit_targ)

        mean_pos_pre = masked_mean(pre_token_log_probs, pre_pos_mask)
        mean_pos_edit = masked_mean(edit_token_log_probs, edit_pos_mask)
        mean_neg_edit = masked_mean(edit_token_log_probs, edit_neg_mask)

        z_per = (mean_pos_edit - mean_neg_edit).sigmoid()
        z_topic_raw = (mean_pos_edit - mean_pos_pre).exp()
        z_topic = min(1, z_topic_raw)

        es_per = z_per * z_topic
        return {
            "acc_per": es_per,
            "z_per": z_per,
            "z_topic": z_topic,
            "z_topic_raw": z_topic_raw,
            "correct_probs": mean_pos_edit,
            "wrong_probs": mean_neg_edit,
        }

def per_generation(
    model,
    tok,
    max_out_len: int,
    target_per, 
    device,
    edited_model=None,
    IKE=False,
    **kwargs
    ):
    def generate_text(query, model, tokenizer):
        input_text = query
        generation_config = {
            "max_new_tokens": max_out_len,
            "temperature": 0,
            "eos_token_id": tokenizer.eos_token_id,
        }
        src_input_ids = tokenizer(input_text).input_ids
        input_ids = torch.tensor([src_input_ids], dtype=torch.long, device=device)
        outputs = model.generate(input_ids, **generation_config)
        response = tokenizer.decode(outputs[0][len(src_input_ids) :], skip_special_tokens=True)
        return response
    
    def clean_text(text):
        return text.strip().split("\n")[0]
    
    if IKE:
        pre_text = clean_text(generate_text(kwargs["pre_q"], model, tok))
        edit_text = clean_text(generate_text(kwargs["edit_q"], model, tok))

    else:
        assert edited_model is not None
        pre_text = clean_text(generate_text(kwargs["inner_q"], model, tok))
        edit_text = clean_text(generate_text(kwargs["inner_q"], edited_model.model, tok))

    ngram_pre_text = n_gram_entropy([pre_text])
    ngram_edit_text = n_gram_entropy([edit_text])
    coherent = ngram_pre_text >= 3.5 and ngram_edit_text >= 3.5
    
    result = {
        "pre_text": pre_text,
        "edit_text": edit_text,
        "ngram_pre_text": ngram_pre_text,
        "ngram_edit_text": ngram_edit_text,
        "coherent": coherent,
        "target_per": target_per,
    }

    return result

def kl_loc_loss(pre, post, mask=None):
    
    pre = pre.to(torch.float32).contiguous()
    post = post[:,-pre.shape[1]:,:].to(torch.float32).contiguous()
    
    sequence = pre.dim() == 3
    pre_ = pre.view(-1, pre.shape[-1])
    post_ = post.view(pre_.shape)
    assert pre_.shape[0] == post_.shape[0]

    if not sequence:
        if pre_.shape[-1] == 1:  # No masking needed for binary classification
            return (pre.sigmoid() * (F.logsigmoid(pre) - F.logsigmoid(post))).mean() + (
                (-pre).sigmoid() * (F.logsigmoid(-pre) - F.logsigmoid(-post))
            ).mean()
    else:  # We have sequences of predictions; masking needed
        # print("sequence")
        if pre_.shape[-1] > 1:
            assert mask is not None
            mask_ = mask.view(pre_.shape[0])
            kl = (pre_.softmax(-1) * (pre_.log_softmax(-1) - post_.log_softmax(-1))).sum(-1)
            return (kl * mask_).sum() / mask_.sum()

    raise NotImplementedError

def F1(model, tok, hparams, prompts, targets, device, locality=False, vanilla_generation=True):
    if vanilla_generation:
        target_new_tokens = tok.encode(targets, add_special_tokens=False)
        prompt_tok = tok(
            prompts,
            return_tensors="pt",
        ).to(device)
        gen_token = model.generate(
            input_ids=prompt_tok['input_ids'],
            attention_mask=prompt_tok['attention_mask'],
            max_new_tokens=len(target_new_tokens),
            pad_token_id=tok.eos_token_id,
            use_cache=False,

        )
        return f1_score(target_new_tokens, gen_token.detach().cpu().numpy().tolist()[0][-len(target_new_tokens):], average='macro')
    if isinstance(prompts, str):
        prompts,targets = [prompts,], [targets,]
    prompt_target = [prompt + ' ' + target for prompt, target in zip(prompts,targets)]
    max_prompt_len = max([len(tok.encode(_)) for _ in prompt_target]) + 1
    prompt_target_tok = tok(
        prompt_target,
        padding=True,
        truncation=True,
        max_length=max(hparams.max_length, max_prompt_len),
        return_tensors="pt",
    ).to(f"cuda:{device}")
    prompt_tok = tok(
        prompts,
        padding=True,
        truncation=True,
        max_length=max(hparams.max_length, max_prompt_len),
        return_tensors="pt",
    )
    num_prompt_toks = [int((i != tok.pad_token_id).sum()) for i in prompt_tok['input_ids']]
    num_pad_toks = [int((i == tok.pad_token_id).sum()) for i in prompt_target_tok['input_ids'].cpu()]
    prompt_len = [x+y for x,y in zip(num_pad_toks,num_prompt_toks)]
    with torch.no_grad():
        outputs = model(**prompt_target_tok)
        if type(outputs) is torch.Tensor:
            logits = outputs
        else:
            logits = outputs.logits
        answers = torch.argmax(logits, dim=-1).squeeze().detach().cpu().numpy().tolist()
        labels = prompt_target_tok['input_ids'].squeeze().detach().cpu().numpy().tolist()
        answers = slice_list(answers,prompt_len,left=True)
        labels = slice_list(labels,prompt_len,left=False)

        return f1_score(answers, labels, average='macro')

def test_instance_change(model, tok, max_length, prompts, targets, device, P = None):
    demo1_str = "Whether FrancoAngeli belongs to category publisher? Yes\nWhether And Other Stories belongs to category people? No\n"
    if P is None:
        prompts = demo1_str +prompts
    else:
        prompts = P + demo1_str + prompts

    if isinstance(prompts, str):
        prompts,targets = [prompts,], [targets,]
    prompt_target = [prompt + ' ' + target for prompt, target in zip(prompts,targets)]
    max_prompt_len = max([len(tok.encode(_)) for _ in prompt_target]) + 1
    prompt_tok = tok(
        prompts,
        padding=True,
        truncation=True,
        max_length=max(max_length, max_prompt_len),
        return_tensors="pt",
    )
    with torch.no_grad():
        pre_edit_outputs = model.generate(
            input_ids=prompt_tok['input_ids'].to(f"cuda:{device}"),
            attention_mask=prompt_tok['attention_mask'].to(f"cuda:{device}"),
            max_new_tokens=2,
            pad_token_id=tok.eos_token_id
        )

        model_response = [tok.decode(x, skip_special_tokens=True) for x in pre_edit_outputs.detach().cpu().numpy().tolist()]
        answer = model_response[0][model_response[0].rfind('?')+2:]
        # print(model_response[0], answer)

        if "yes" in answer.lower():
            return np.ones(1)
        else:
            if "no" not in answer.lower():
                print(f"entity error in define yes or no: {answer}")
                return np.array([-1.0])
            return np.zeros(1)

def test_concept_gen(model, tok, max_length, prompts, targets, device):
    if isinstance(prompts, str):
        prompts,targets = [prompts,], [targets,]
    prompts = [prompt + ' ' for prompt in prompts]
    prompt_target = [prompt + ' ' + target for prompt, target in zip(prompts,targets)]
    max_prompt_len = max([len(tok.encode(_)) for _ in prompt_target]) + 1
    prompt_tok = tok(
        prompts,
        padding=True,
        truncation=True,
        max_length=max(max_length, max_prompt_len),
        return_tensors="pt",
    )
    with torch.no_grad():
        pre_edit_outputs = model.generate(
            input_ids=prompt_tok['input_ids'].to(f"cuda:{device}"),
            attention_mask=prompt_tok['attention_mask'].to(f"cuda:{device}"),
            max_new_tokens=40,
            pad_token_id=tok.eos_token_id
        )

        model_response = [tok.decode(x, skip_special_tokens=True) for x in pre_edit_outputs.detach().cpu().numpy().tolist()]
        answer = model_response[0][len(prompts[0]):]
        return answer


def test_safety_gen(
        model, 
        tokenizer, 
        test_prompt, 
        cuda,
        max_tokens = 1624, 
        max_output_tokens=600):
    tokenizer.padding_side = 'left'
    # if input_tokens (at least 1024) + output_tokens (at least 600) < 1624, truncate the input length (from right to left, as harmful questions typically appear on the right)
    if max_tokens < 1624:
        only_response = []
        for item in test_prompt:
            input = tokenizer([item,], return_tensors="pt", padding=True, truncation=True).to(f"cuda:{cuda}")
            if input["input_ids"].size(-1) > max_tokens-max_output_tokens:
                input = {k: v[:, -(max_tokens - max_output_tokens):] for k, v in input.items()}
            with torch.no_grad():
                outputs = model.generate(**input, max_new_tokens=max_output_tokens)
                texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
                texts = texts[0]
            if input["input_ids"].size(-1) > max_tokens-max_output_tokens:
                max_overlap_len = min(len(item), len(texts))
                overlap = next((item[-i:] for i in range(max_overlap_len, 0, -1) if item[-i:] == texts[:i]), "")
            else:
                overlap = item
            only_response.append(texts[len(overlap)+1:].lstrip())
        return only_response
    else:
        input = tokenizer(test_prompt, return_tensors="pt", padding=True, truncation=True).to(f"cuda:{cuda}")
        with torch.no_grad():
            outputs = model.generate(**input, max_new_tokens=max_output_tokens)
            texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
            only_response = [out[len(test_prompt[index])+1:] for index, out in enumerate(texts)]
        return only_response


In [ ]:
# evaluate.py

"""
Contains evaluation utilities for pytorch-based rewriting methods.
To use, simply call `compute_rewrite_quality_zsre` with the
appropriate arguments, which returns a dictionary containing them.
"""
from easyeditor.models.melo.melo import LORA

import typing
from itertools import chain
from typing import List, Optional

import numpy as np
import torch
# from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoTokenizer
from easyeditor.util import HyperParams
from easyeditor.evaluate.evaluate_utils import (
    test_seq2seq_batch_prediction_acc, 
    test_batch_prediction_acc, 
    test_prediction_acc,
    test_prediction_acc_LLM_judge,
    # test_generation_quality, 
    test_concept_gen,
    test_safety_gen,
    test_instance_change,
    PPL,
    OOD_PPL,
    kl_loc_loss,
    es,
    es_per_icl,
    per_generation,
    F1
)

def compute_edit_quality(
    model,
    model_name,
    hparams: HyperParams,
    tok: AutoTokenizer,
    record: typing.Dict,
    device,
    eval_metric: str = 'token_em',
    test_generation = False
) -> typing.Dict:
    """
    Given a rewritten model, computes generalization and specificity metrics for
    the desired rewrite (passed in via the CounterFact dataset record). Returns a
    dictionary containing those metrics.

    :param model: Rewritten model
    :param tok: Tokenizer
    :param record: CounterFact dataset record
    :paran snips: ???
    :param vec: ???
    :return: Dictionary containing rewriting metrics
    """
    if isinstance(model,LORA):
        model=model.model
    # First, unpack rewrite evaluation record.
    target_new, ground_truth = (
        record[x] for x in ["target_new", "ground_truth"]
    )

    rewrite_prompts = record["prompt"]
    rephrase_prompts = record["rephrase_prompt"] if 'rephrase_prompt' in record.keys() else None
    ret = compute_rewrite_or_rephrase_quality(model, model_name, hparams, tok,
                                              rewrite_prompts, target_new, device=device, eval_metric=eval_metric)

    ret['locality'] = {}
    ret['portability'] = {}
    if rephrase_prompts is not None:
        ret.update(
            compute_rewrite_or_rephrase_quality(model, model_name, hparams, tok,
                                                rephrase_prompts, target_new, device=device, test_rephrase=True, eval_metric=eval_metric)
        )

    if 'locality' in record.keys() and any(record['locality']):
        for locality_key in record['locality'].keys():
            ret['locality'].update(
                compute_locality_quality(model, model_name, hparams, tok, locality_key,
                                         record['locality'][locality_key]['prompt'],
                                         record['locality'][locality_key]['ground_truth'], device=device)
            )
    if 'portability' in record.keys() and any(record['portability']):
        for portability_key in record['portability'].keys():
            ret['portability'].update(
                compute_portability_quality(model, model_name, hparams, tok, portability_key,
                                            record['portability'][portability_key]['prompt'],
                                            record['portability'][portability_key]['ground_truth'], device=device)
            )
    if test_generation:
        if hparams.alg_name == 'GRACE':
            ret['fluency'] = test_generation_quality(model=model,tok=tok,prefixes=rewrite_prompts if isinstance(rewrite_prompts,list) else [rewrite_prompts,], max_out_len=100, vanilla_generation=True)
        else:
            ret['fluency'] = test_generation_quality(model=model,tok=tok,prefixes=rewrite_prompts if isinstance(rewrite_prompts,list) else [rewrite_prompts,], max_out_len=100, vanilla_generation=False)
    return ret

def compute_rewrite_or_rephrase_quality(
    model,
    model_name,
    hparams: HyperParams,
    tok: AutoTokenizer,
    prompt: str,
    target_new: str,
    device,
    test_rephrase: bool = False,
    eval_metric: str = 'token_em'
) -> typing.Dict:
    
    if not test_rephrase:
        key = 'rewrite'
    else:
        key = 'rephrase'
    # using real-world evaluation: autoregressive decoding, natural stop criteria, LLM-as-a-Judge
    if hasattr(hparams, 'evaluation_type') and hparams.evaluation_type == "LLM-judge":
        acc, gen_content = test_prediction_acc_LLM_judge(model, tok, hparams, prompt, target_new, device, locality=False)
        ret = {
            f"{key}_acc": acc,
            f"{key}_gen_content": gen_content
        }
    elif hasattr(hparams, 'evaluation_type') and hparams.evaluation_type == "generate-text":
        gen_content_model = test_prediction_acc_LLM_judge(model, tok, hparams, prompt, target_new, device, locality=False)
        ret = {
            f"{key}_gen_content": gen_content_model
        }
    else:  # traditional evaluation 
        if eval_metric == 'ppl':
            ppl = PPL(model, tok, prompt, target_new, device)
            ret = {
                f"{key}_ppl": ppl
            }
        elif eval_metric == 'ood_ppl':
            ans = OOD_PPL(model, tok, prompt, target_new, device)
            ret = {
                f"ood_acc": ans
            }
        elif hparams.alg_name=="GRACE":
            # ppl = PPL(model, tok, prompt, target_new, device)
            if 't5' in model_name.lower():
                acc = test_seq2seq_batch_prediction_acc(model, tok, hparams, prompt, target_new, device)
            else:
                acc = test_prediction_acc(model, tok, hparams, prompt, target_new, device, vanilla_generation=True)
            f1 = F1(model,tok,hparams,prompt,target_new,device, vanilla_generation=True)
            ret = {
                f"{key}_acc": acc,
                # f"{key}_PPL": ppl,
                f"{key}_F1":f1     
            }        
        else:  # teacher-forcing evaluation
            if 't5' in model_name.lower():
                acc = test_seq2seq_batch_prediction_acc(model, tok, hparams, prompt, target_new, device)
            else:
                acc = test_prediction_acc(model, tok, hparams, prompt, target_new, device)
            ret = {
                f"{key}_acc": acc
            }
    return ret

def compute_locality_quality(
    model,
    model_name,
    hparams: HyperParams,
    tok: AutoTokenizer,
    locality_key: str,
    prompt: typing.Union[str, List[str]],
    locality_ground_truth: typing.Union[str, List[str]],
    device,
) -> typing.Dict:

    # using real-world evaluation: autoregressive decoding, natural stop criteria, LLM-as-a-Judge
    if hasattr(hparams, 'evaluation_type'):
        loc_tokens = test_prediction_acc_LLM_judge(model, tok, hparams, prompt, locality_ground_truth, device, locality=True)
    else:  # traditional evaluation 
        if 't5' in model_name.lower():
            loc_tokens = test_seq2seq_batch_prediction_acc(model, tok, hparams, prompt, locality_ground_truth, device, locality=True)
        else:
            loc_tokens = test_prediction_acc(model, tok, hparams, prompt, locality_ground_truth, device, locality=True, vanilla_generation=hparams.alg_name=='GRACE')
        if type(loc_tokens) is not list:
            loc_tokens = [loc_tokens,]

    ret = {
        f"{locality_key}_output": loc_tokens
    }
    return ret

def compute_portability_quality(
    model,
    model_name,
    hparams: HyperParams,
    tok: AutoTokenizer,
    portability_key: str,
    prompt: typing.Union[str, List[str]],
    ground_truth: typing.Union[str, List[str]],
    device,
) -> typing.Dict:
    # using real-world evaluation: autoregressive decoding, natural stop criteria, LLM-as-a-Judge
    if hasattr(hparams, 'evaluation_type') and hparams.evaluation_type == "LLM-judge":
        portability_correct = test_prediction_acc_LLM_judge(model, tok, hparams, prompt, ground_truth, device, locality=False)
    elif hasattr(hparams, 'evaluation_type') and hparams.evaluation_type == "generate-text":
        portability_correct = test_prediction_acc_LLM_judge(model, tok, hparams, prompt, ground_truth, device, locality=False)
    else:  # traditional evaluation
        if 't5' in model_name.lower():
            portability_correct = test_seq2seq_batch_prediction_acc(model, tok, hparams, prompt, ground_truth, device)
        else:
            portability_correct = test_prediction_acc(model, tok, hparams, prompt, ground_truth, device, vanilla_generation=hparams.alg_name=='GRACE')

    ret = {
        f"{portability_key}_acc": portability_correct
    }
    return ret

def compute_icl_edit_quality(
        model,
        model_name,
        hparams: HyperParams,
        tok: AutoTokenizer,
        icl_examples,
        record: typing.Dict,
        device,
        pre_edit: bool = False,
        test_generation = False
) -> typing.Dict:
    """
    Given a rewritten model, computes generalization and specificity metrics for
    the desired rewrite (passed in via the CounterFact dataset record). Returns a
    dictionary containing those metrics.

    :param model: Rewritten model
    :param tok: Tokenizer
    :param record: CounterFact dataset record
    :param snips: ???
    :param vec: ???
    :return: Dictionary containing rewriting metrics
    """

    # First, unpack rewrite evaluation record.
    target_new, ground_truth = (
        record[x] for x in ["target_new", "ground_truth"]
    )
    prompt = record["prompt"]
    rephrase = record["rephrase_prompt"] if 'rephrase_prompt' in record.keys() else None
    new_fact = f'New Fact: {prompt} {target_new}\nPrompt: {prompt}'

    if pre_edit:
        edit_acc = icl_lm_eval(model, model_name, hparams, tok, icl_examples,
                               target_new, prompt)
    else:
        edit_acc = icl_lm_eval(model, model_name, hparams, tok, icl_examples,
                               target_new, new_fact)
    ret = {
        f"rewrite_acc": [edit_acc]
    }
    ret['locality'] = {}
    ret['portability'] = {}
    if rephrase is not None:
        rephrase_acc = icl_lm_eval(model, model_name, hparams, tok, icl_examples,
                                   target_new, f'New Fact: {prompt} {target_new}\nPrompt: {rephrase}')
        ret['rephrase_acc'] = rephrase_acc

    if 'locality' in record.keys() and any(record['locality']):
        for locality_key in record['locality'].keys():
            if isinstance(record['locality'][locality_key]['ground_truth'], list):
                pre_neighbor = []
                post_neighbor = []
                for x_a, x_p in zip(record['locality'][locality_key]['ground_truth'],
                                    record['locality'][locality_key]['prompt']):
                    tmp_pre_neighbor = icl_lm_eval(model, model_name, hparams, tok, [''], x_a,
                                                   f"{x_p}", neighborhood=True)
                    tmp_post_neighbor = icl_lm_eval(model, model_name, hparams, tok, icl_examples, x_a,
                                                    f"New Fact: {prompt} {target_new}\nPrompt: {x_p}",
                                                    neighborhood=True)
                    if type(tmp_pre_neighbor) is not list:
                        tmp_pre_neighbor = [tmp_pre_neighbor, ]
                    if type(tmp_post_neighbor) is not list:
                        tmp_post_neighbor = [tmp_post_neighbor, ]
                    assert len(tmp_pre_neighbor) == len(tmp_post_neighbor)
                    pre_neighbor.append(tmp_pre_neighbor)
                    post_neighbor.append(tmp_post_neighbor)
                res = []
                for ans, label in zip(pre_neighbor, post_neighbor):
                    temp_acc = np.mean(np.equal(ans, label))
                    if np.isnan(temp_acc):
                        continue
                    res.append(temp_acc)
                ret['locality'][f'{locality_key}_acc'] = res
            else:
                pre_neighbor = icl_lm_eval(model, model_name, hparams, tok, [''],
                                           record['locality'][locality_key]['ground_truth'],
                                           f"{record['locality'][locality_key]['prompt']}",
                                           neighborhood=True)
                post_neighbor = icl_lm_eval(model, model_name, hparams, tok, icl_examples,
                                            record['locality'][locality_key]['ground_truth'],
                                            f"New Fact: {prompt} {target_new}\nPrompt: {record['locality'][locality_key]['prompt']}",
                                            neighborhood=True)
                if type(pre_neighbor) is not list:
                    pre_neighbor = [pre_neighbor, ]
                if type(post_neighbor) is not list:
                    post_neighbor = [post_neighbor, ]
                assert len(pre_neighbor) == len(post_neighbor)

                ret['locality'][f'{locality_key}_acc'] = np.mean(np.equal(pre_neighbor, post_neighbor))
    # Form a list of lists of prefixes to test.
    if 'portability' in record.keys() and any(record['portability']):
        for portability_key in record['portability'].keys():
            if pre_edit:
                icl_input = ['']
                x_prefix = ""
            else:
                icl_input = icl_examples
                x_prefix = f"New Fact: {prompt} {target_new}\nPrompt: "
            if isinstance(record['portability'][portability_key]['ground_truth'], list):
                portability_acc = []
                for x_a, x_p in zip(record['portability'][portability_key]['ground_truth'],
                                    record['portability'][portability_key]['prompt']):
                    tmp_portability_acc = icl_lm_eval(model, model_name, hparams, tok, icl_input, x_a,
                                                      f"{x_prefix}{x_p}")
                portability_acc.append(tmp_portability_acc)
            else:
                portability_acc = icl_lm_eval(model, model_name, hparams, tok, icl_input,
                                              record['portability'][portability_key]['ground_truth'],
                                              f"{x_prefix}{record['portability'][portability_key]['prompt']}")
            ret['portability'][f'{portability_key}_acc'] = portability_acc

    if test_generation:
        ret['fluency'] = test_generation_quality(model=model,tok=tok, prefixes=new_fact if isinstance(new_fact,list) else [new_fact,], max_out_len=100, vanilla_generation=False)
    return ret

def icl_lm_eval(
        model,
        model_name,
        hparams: HyperParams,
        tokenizer,
        icl_examples,
        target,
        x,
        neighborhood=False
)-> typing.Dict:
    device = torch.device(f'cuda:{hparams.device}')
    if 't5' in model_name.lower():
        target_len = len(tokenizer.encode(target))
        target_ids = tokenizer(f'{x} {target}', return_tensors='pt')['input_ids'].to(device)
        encodings = tokenizer(''.join(icl_examples), return_tensors='pt')
        input_ids = encodings['input_ids'].to(device)
        attention_mask = encodings['attention_mask'].to(device)
        with torch.no_grad():
            logits = model(input_ids=input_ids, attention_mask=attention_mask, labels=target_ids).logits
            ans = torch.argmax(logits, dim=-1)[:,-target_len:-1].squeeze()
            target_ids = target_ids[:,-target_len:-1]
            if neighborhood:
                return ans.squeeze().detach().cpu().numpy().tolist()
            return torch.mean((ans == target_ids.to(ans.device).squeeze()).float(), dim=-1).detach().cpu().numpy().tolist()
    elif 'llama' in model_name.lower():
        target_ids = tokenizer(target, return_tensors='pt')['input_ids'].to(device)
        encodings = tokenizer(''.join(icl_examples) + f'{x} {target}', return_tensors='pt')
        input_ids = encodings['input_ids'].to(device)
        attention_mask = encodings['attention_mask'].to(device)
        logits = model(input_ids=input_ids, attention_mask=attention_mask).logits
        ans = torch.argmax(logits, dim=-1)[:,-target_ids.size(1):-1].squeeze()
        target_ids = target_ids[:,1:]
        if neighborhood:
            return ans.squeeze().detach().cpu().numpy().tolist()
        return torch.mean((ans == target_ids.to(ans.device).squeeze()).float(), dim=-1).detach().cpu().numpy().tolist()
    else:
        target_ids = tokenizer(' ' + target + '\n', return_tensors='pt')['input_ids'].to(device)
        encodings = tokenizer(''.join(icl_examples) + f'{x} {target}', return_tensors='pt')
        input_ids = encodings['input_ids'].to(device)
        attention_mask = encodings['attention_mask'].to(device)
        logits = model(input_ids=input_ids, attention_mask=attention_mask).logits
        ans = torch.argmax(logits, dim=-1)[:,-target_ids.size(1):-1].squeeze()
        target_ids = target_ids[:,:-1]
        if neighborhood:
            return ans.squeeze().detach().cpu().numpy().tolist()
        return torch.mean((ans == target_ids.to(ans.device).squeeze()).float(), dim=-1).detach().cpu().numpy().tolist()


In [ ]:
# editor.py

from typing import Optional, Union, List, Tuple, Dict
from time import time
from tqdm import tqdm
import json
import torch
import numpy as np
import random
from easyeditor.models.melo.melo import LORA
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, BitsAndBytesConfig
from transformers import LlamaTokenizer,PreTrainedTokenizerFast, LlamaTokenizerFast
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import GPT2TokenizerFast, GPT2Tokenizer
from easyeditor.util.globals import *
from easyeditor.editors.utils import _chunks, _prepare_requests, summary_metrics
from easyeditor.editors.batch_editor import BatchEditor
# from easyeditor.evaluate import compute_edit_quality, compute_icl_edit_quality, compute_sent_metric
from easyeditor.util import nethook
from easyeditor.util.hparams import HyperParams
from easyeditor.util.alg_dict import *
# from ..evaluate.evaluate_utils import test_generation_quality

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)

LOG = logging.getLogger(__name__)
def make_logs():

    f_h, s_h = get_handler('logs', log_name='run.log')
    LOG.addHandler(f_h)
    LOG.addHandler(s_h)

def seed_everything(seed):
    if seed >= 10000:
        raise ValueError("seed number should be less than 10000")
    if torch.distributed.is_initialized():
        rank = torch.distributed.get_rank()
    else:
        rank = 0
    seed = (rank * 100000) + seed

    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    
seed_everything(42)
  
class BaseEditor:
    """Base editor for all methods"""

    @classmethod
    def from_hparams(cls, hparams: HyperParams):
        return cls(hparams)

    def __init__(self, hparams: HyperParams):
        assert hparams is not None, 'Error: hparams is None.'
        self.model_name = hparams.model_name
        self.apply_algo = ALG_DICT[hparams.alg_name]
        self.alg_name = hparams.alg_name
        make_logs()
        LOG.info("Instantiating model")

        if type(self.model_name) is str:
            device_map = 'auto' if hparams.model_parallel else None
            torch_dtype = torch.float16 if hasattr(hparams, 'fp16') and hparams.fp16 else torch.float32
            
            # QLoRA configuration
            if hparams.alg_name == 'QLoRA':
                bnb_config = BitsAndBytesConfig(
                    load_in_4bit=(hparams.quantization_bit == 4),
                    bnb_4bit_use_double_quant=hparams.double_quant,
                    bnb_4bit_quant_type=hparams.quant_type,
                    bnb_4bit_compute_dtype=torch.bfloat16
                )
                model_kwargs = {
                    "quantization_config": bnb_config,
                    "torch_dtype": torch_dtype,
                    "device_map": {'': hparams.device}
                }
            else:
                model_kwargs = {
                    "torch_dtype": torch_dtype,
                    "device_map": device_map
                }

            if 't5' in self.model_name.lower():
                self.model = T5ForConditionalGeneration.from_pretrained(self.model_name, **model_kwargs)
                self.tok = T5Tokenizer.from_pretrained(self.model_name)
            elif 'chatglm-api' in self.model_name.lower():
                self.model, self.tok = None, None
                self.hparams = hparams
                return
            elif 'gpt-3.5' in self.model_name.lower():
                self.model, self.tok = None, None
            elif 'gpt' in self.model_name.lower():
                self.model = AutoModelForCausalLM.from_pretrained(self.model_name, **model_kwargs)
                self.tok = GPT2Tokenizer.from_pretrained(self.model_name)
                self.tok.pad_token_id = self.tok.eos_token_id
            elif 'llama' in self.model_name.lower():
                self.model = AutoModelForCausalLM.from_pretrained(self.model_name, **model_kwargs)
                self.tok = AutoTokenizer.from_pretrained(self.model_name)
                self.tok.pad_token_id = self.tok.eos_token_id
            elif 'baichuan' in self.model_name.lower():
                self.model = AutoModelForCausalLM.from_pretrained(self.model_name, **model_kwargs, trust_remote_code=True)
                self.tok = AutoTokenizer.from_pretrained(self.model_name,trust_remote_code=True)
                self.tok.pad_token_id = self.tok.eos_token_id
            elif 'chatglm' in self.model_name.lower():
                self.model = AutoModel.from_pretrained(self.model_name,trust_remote_code=True, **model_kwargs)
                self.tok = AutoTokenizer.from_pretrained(self.model_name,trust_remote_code=True)
                if 'chatglm2'in self.model_name.lower(): 
                    self.tok.unk_token_id = 64787
                else: 
                    self.tok.pad_token_id = self.tok.eos_token_id
            elif 'internlm' in self.model_name.lower():
                self.model = AutoModel.from_pretrained(self.model_name,trust_remote_code=True, **model_kwargs)
                self.tok = AutoTokenizer.from_pretrained(self.model_name,trust_remote_code=True)
                self.tok.pad_token_id = self.tok.eos_token_id
            elif 'qwen2' in self.model_name.lower():
                self.model = AutoModelForCausalLM.from_pretrained(self.model_name,trust_remote_code=True, torch_dtype=torch_dtype if hparams.alg_name not in ['MEND'] else torch.bfloat16, device_map=device_map)
                self.tok = AutoTokenizer.from_pretrained(self.model_name, eos_token='<|endoftext|>', pad_token='<|endoftext|>',unk_token='<|endoftext|>', trust_remote_code=True)
            elif 'qwen' in self.model_name.lower():
                self.model = AutoModelForCausalLM.from_pretrained(self.model_name,fp32=False,trust_remote_code=True, **model_kwargs)
                self.tok = AutoTokenizer.from_pretrained(self.model_name, eos_token='<|endoftext|>', pad_token='<|endoftext|>',unk_token='<|endoftext|>', trust_remote_code=True)
            elif 'mistral' in self.model_name.lower():
                self.model = AutoModelForCausalLM.from_pretrained(self.model_name, **model_kwargs)
                self.tok = AutoTokenizer.from_pretrained(self.model_name)
                self.tok.pad_token_id = self.tok.eos_token_id
            else:
                raise NotImplementedError

            if self.tok is not None and (isinstance(self.tok, GPT2Tokenizer) or isinstance(self.tok, GPT2TokenizerFast) or isinstance(self.tok, LlamaTokenizer) or isinstance(self.tok, LlamaTokenizerFast) or isinstance(self.tok, PreTrainedTokenizerFast)) and (hparams.alg_name not in ['ROME', 'MEMIT', 'EMMET', 'R-ROME','AlphaEdit','CORE']):
                LOG.info('AutoRegressive Model detected, set the padding side of Tokenizer to left...')
                self.tok.padding_side = 'left'
            if self.tok is not None and ('mistral' in self.model_name.lower() or 'llama' in self.model_name.lower() or 'qwen' in self.model_name.lower()) and (hparams.alg_name in ['ROME', 'MEMIT', 'EMMET', 'R-ROME','AlphaEdit', 'CORE']):
                LOG.info('AutoRegressive Model detected, set the padding side of Tokenizer to right...')
                self.tok.padding_side = 'right'
        else:
            self.model, self.tok = self.model_name

        if hparams.model_parallel: 
            hparams.device = str(self.model.device).split(":")[1]
        if not hparams.model_parallel and hasattr(hparams, 'device') and hparams.alg_name != 'QLoRA':
            self.model.to(f'cuda:{hparams.device}')

        self.hparams = hparams

    def edit(self,
             prompts: Union[str, List[str]],
             target_new: Union[str, List[str]],
             ground_truth: Optional[Union[str, List[str]]] = None,
             target_neg: Optional[Union[str, List[str]]] = None,
             rephrase_prompts: Optional[Union[str, List[str]]] = None,
             locality_inputs:  Optional[Dict] = None,
             portability_inputs: Optional[Dict] = None,
             sequential_edit=False,
             verbose=True,
             **kwargs
             ):
        """
        `prompts`: list or str
            the prompts to edit
        `ground_truth`: str
            the ground truth / expected output
        `locality_inputs`: dict
            for locality
        """
        test_generation = kwargs.pop('test_generation', False)

        if isinstance(prompts, List):
            assert len(prompts) == len(target_new)
        else:
            prompts, target_new = [prompts,], [target_new,]

        if hasattr(self.hparams, 'batch_size') and not BatchEditor.is_batchable_method(self.alg_name):  # For Singleton Editing, bs=1
            assert self.hparams.batch_size == 1, 'Single Editing: batch_size should be set to 1'

        if ground_truth is not None:
            ground_truth = [ground_truth,] if isinstance(ground_truth, str) else ground_truth
        else:# Default ground truth is <|endoftext|>
            ground_truth = ['<|endoftext|>'] * (len(prompts))

        if "requests" in kwargs.keys():
            requests = kwargs["requests"]
        else:
            requests = _prepare_requests(prompts, target_new, ground_truth, target_neg, rephrase_prompts, locality_inputs, portability_inputs, **kwargs)

        return self.edit_requests(requests, sequential_edit, verbose, test_generation=test_generation, **kwargs)

    def batch_edit(self,
                   prompts: List[str],
                   target_new: List[str],
                   ground_truth: Optional[List[str]] = None,
                   target_neg: Optional[List[str]] = None,
                   rephrase_prompts: Optional[List[str]] = None,
                   locality_inputs: Optional[Dict] = None,
                   portability_inputs: Optional[Dict] = None,
                   sequential_edit=False,
                   verbose=True,
                   **kwargs
                   ):
        """
        `prompts`: list or str
            the prompts to edit
        `ground_truth`: str
            the ground truth / expected output
        """
        assert len(prompts) == len(target_new)
        test_generation = kwargs['test_generation'] if 'test_generation' in kwargs.keys() else False
        if ground_truth is not None:
            if isinstance(ground_truth, str):
                ground_truth = [ground_truth,]
            else:
                assert len(ground_truth) == len(prompts)
        else: # Default ground truth is <|endoftext|>
            ground_truth = ['<|endoftext|>' for _ in range(len(prompts))]


        assert BatchEditor.is_batchable_method(self.alg_name), f'The Method {self.alg_name} can not batch edit examples.'

        requests = _prepare_requests(prompts, target_new, ground_truth, target_neg, rephrase_prompts, locality_inputs, portability_inputs, **kwargs)

        assert hasattr(self.hparams, 'batch_size'), f'Method {self.alg_name} found, pls specify the batch_size....'
        all_metrics = []
        for record_chunks in _chunks(requests, self.hparams.batch_size):
            start = time()

            edited_model, weights_copy = self.apply_algo(
                self.model,
                self.tok,
                record_chunks,
                self.hparams,
                copy=False,
                return_orig_weights=True
            )
            exec_time = time() - start
            LOG.info(f"Execution editing took {exec_time}")

            start = time()
            chunk_metrics = []
            for i, request in enumerate(record_chunks):

                metrics = {
                    'case_id': i,
                    "requested_rewrite": request,
                    "time": exec_time,
                    "post": compute_edit_quality(edited_model, self.model_name, self.hparams, self.tok, request, self.hparams.device, test_generation=test_generation),
                }

                chunk_metrics.append(metrics)

            if sequential_edit:
                self.model = edited_model
            else:
                if self.alg_name == 'KN' or self.alg_name == 'GRACE' or self.alg_name == 'WISE':
                    with torch.no_grad():
                        weights_copy()
                elif self.alg_name == 'LoRA' or self.alg_name == 'QLoRA' or self.alg_name == 'DPO':
                    edited_model.unload()
                    del self.model.peft_config
                elif self.alg_name == 'MELO':
                    self.model = edited_model
                else:
                    with torch.no_grad():
                        for k, v in weights_copy.items():
                            nethook.get_parameter(self.model, k)[...] = v.to(f"cuda:{self.hparams.device}")

            for i, request in enumerate(record_chunks):
                chunk_metrics[i]["pre"] = compute_edit_quality(self.model, self.model_name, self.hparams, self.tok, request, self.hparams.device, test_generation=test_generation)

                if verbose:
                    LOG.info(
                        f"{i} editing: {request['prompt']} -> {request['target_new']}  \n {chunk_metrics[i]}"
                    )

            LOG.info(f"Evaluation took {time() - start}")
            all_metrics.extend(chunk_metrics)
        return all_metrics, edited_model, weights_copy

    def edit_requests(self,
             requests,
             sequential_edit=False,
             verbose=True,
             test_generation=False,
             **kwargs
             ):
        """
        `prompts`: list or str
            the prompts to edit
        `ground_truth`: str
            the ground truth / expected output
        `locality_inputs`: dict
            for locality
        """
        eval_metric= kwargs['eval_metric'] if 'eval_metric' in kwargs.keys() else 'exact match'
        if hasattr(self.hparams, 'batch_size'):  # For Singleton Editing, bs=1
            assert self.hparams.batch_size == 1, 'Single Editing: batch_size should be set to 1'
        all_metrics = []
        if 'pre_edit' in kwargs and kwargs['pre_edit'] is not None:
            metrics = kwargs['pre_edit']
            all_metrics = metrics
        else:
            for i, request in enumerate(tqdm(requests)):
                if self.alg_name == 'IKE':
                    assert 'train_ds' in kwargs.keys(), print('IKE need train_ds(For getting In-Context prompt)')
                    metrics = {"pre": compute_icl_edit_quality(self.model, self.model_name, self.hparams, self.tok, [''], request, self.hparams.device, pre_edit=True)}
                else:
                    metrics = {"pre": compute_edit_quality(self.model, self.model_name, self.hparams, self.tok, request, self.hparams.device, eval_metric=eval_metric, test_generation=test_generation)}
                all_metrics.append(metrics)
            if 'pre_file' in kwargs and kwargs['pre_file'] is not None:
                json.dump(all_metrics, open(kwargs['pre_file'], 'w'), indent=4)

        def edit_func(request):
            if self.alg_name == 'IKE' or self.alg_name == 'ICE':
                edited_model, weights_copy, icl_examples = self.model, {}, self.apply_algo(
                    self.model,
                    self.tok,
                    [request],
                    self.hparams,
                    copy=False,
                    return_orig_weights=True,
                    keep_original_weight=False,
                    train_ds=kwargs['train_ds'] if self.alg_name == 'IKE' else None
                )
            else:
                edited_model, weights_copy = self.apply_algo(
                    self.model,
                    self.tok,
                    [request],
                    self.hparams,
                    copy=False,
                    return_orig_weights=True,
                    keep_original_weight=False,
                    train_ds=kwargs['train_ds'] if self.alg_name == 'IKE' else None
                )
                icl_examples = None
            return edited_model, weights_copy, icl_examples

        def edit_evaluation(all_metrics, request, edited_model, idx, test_generation, icl_examples, **kwargs):
            eval_metric= kwargs['eval_metric'] if 'eval_metric' in kwargs.keys() else 'exact match'
            if self.alg_name == 'IKE':
                all_metrics[idx].update({
                    'case_id': idx,
                    "requested_rewrite": request,
                    "post": compute_icl_edit_quality(self.model, self.model_name, self.hparams, self.tok, icl_examples, request, self.hparams.device ,test_generation=test_generation),
                })
                if "metric_kwargs" in kwargs:
                    all_metrics[idx].update(compute_sent_metric(self.model, edited_model, self.model_name, self.hparams, self.tok,metric_kwargs=kwargs["metric_kwargs"][idx], device=self.hparams.device))
    
            else:
                all_metrics[idx].update({
                    'case_id': idx,
                    "requested_rewrite": request,
                    "post": compute_edit_quality(edited_model, self.model_name, self.hparams, self.tok, request, self.hparams.device, eval_metric=eval_metric, test_generation=test_generation),
                })
                if "metric_kwargs" in kwargs:
                    all_metrics[idx].update(compute_sent_metric(self.model, edited_model, self.model_name, self.hparams, self.tok,metric_kwargs=kwargs["metric_kwargs"][idx], device=self.hparams.device))
                if 'locality' in all_metrics[idx]['post'].keys() and not hasattr(self.hparams, 'evaluation_type'):
                    for locality_key in request['locality'].keys():
                        locality_result = []
                        if hasattr(self.hparams, 'evaluation_type'):
                            locality_result.append(float(all_metrics[idx]['post']['locality'][f'{locality_key}_output']==all_metrics[idx]['pre']['locality'][f'{locality_key}_output']))
                        else:
                            for ans, label in zip(all_metrics[idx]['post']['locality'][f'{locality_key}_output'], all_metrics[idx]['pre']['locality'][f'{locality_key}_output']):
                                locality_result.append(np.mean(np.equal(ans, label)))
                        all_metrics[idx]['post']['locality'][f'{locality_key}_acc'] = locality_result
                        all_metrics[idx]['post']['locality'].pop(f'{locality_key}_output')
                    all_metrics[idx]['pre'].pop('locality')

            if verbose:
                LOG.info(f"{idx} editing: {request['prompt']} -> {request['target_new']}  \n\n {all_metrics[idx]}")


        if sequential_edit:
            for i, request in enumerate(tqdm(requests, total=len(requests))):
                edited_model, weights_copy, icl_examples = edit_func(request)
            if self.alg_name == 'LoRA' or self.alg_name == 'QLoRA' or self.alg_name == 'DPO':
                self.model = edited_model
            if self.alg_name == 'WISE' and hasattr(self.hparams, 'save_path') and self.hparams.save_path:
                print("Start saving the WISE model!")
                edited_model.save(self.hparams.save_path)
            for i, request in enumerate(requests):
                edit_evaluation(all_metrics, request, edited_model, i, test_generation, icl_examples, **kwargs)
        else:
            for i, request in enumerate(tqdm(requests, total=len(requests))):
                edited_model, weights_copy, icl_examples = edit_func(request)
                edit_evaluation(all_metrics, request, edited_model, i, test_generation, icl_examples, **kwargs)
                if self.alg_name == 'KN' or self.alg_name == 'GRACE' or self.alg_name == 'WISE':
                    with torch.no_grad():
                        weights_copy()
                elif self.alg_name == 'LoRA' or self.alg_name == 'QLoRA' or self.alg_name == 'DPO':
                    edited_model.unload()
                    del self.model.peft_config
                elif self.alg_name == 'MELO':
                    self.model = edited_model
                else:
                    with torch.no_grad():
                        for k, v in weights_copy.items():
                            nethook.get_parameter(self.model, k)[...] = v.to(f"cuda:{self.hparams.device}")


        if isinstance(edited_model, LORA):
            edited_model = edited_model.model
        if not hasattr(self.hparams, 'evaluation_type') or self.hparams.evaluation_type != "generate-text":
            summary_metrics(all_metrics)

        return all_metrics, edited_model, weights_copy

    def normal_edit(
        self,
        prompts: List[str],
        target_new: List[str],
        sequential_edit=False,
    ):
        """
        `prompts`: list or str
            the prompts to edit
        `ground_truth`: str
            the ground truth / expected output
        """
        assert len(prompts) == len(target_new)
        ground_truth = ['<|endoftext|>' for _ in range(len(prompts))]


        assert BatchEditor.is_batchable_method(self.alg_name), f'The Method {self.alg_name} can not batch edit examples.'

        requests = _prepare_requests(prompts, target_new, ground_truth)

        assert hasattr(self.hparams, 'batch_size'), f'Method {self.alg_name} found, pls specify the batch_size....'

        # print(f"[editor.py][batch_edit] `batch_size`={self.hparams.batch_size}")
        # for epc in range(epoch):
        #     print(f"[editor.py][batch_edit] `Epoch` = {epc+1}")
        #     for record_chunks in self._chunks(requests, self.hparams.batch_size):
        start = time()

        edited_model, weights_copy = self.apply_algo(
            self.model,
            self.tok,
            requests,  # record_chunks -> requests
            self.hparams,
            copy=False,
            return_orig_weights=True,
            keep_original_weight=False,
        )
        exec_time = time() - start
        LOG.info(f"Execution editing took {exec_time}")

        with torch.no_grad():
            for k, v in weights_copy.items():
                nethook.get_parameter(self.model, k)[...] = v.to(f"cuda:{self.hparams.device}")

        return None, edited_model, weights_copy
    
    def generate_edit(
        self,
        prompts: Union[str, List[str]],
        target_new: Union[str, List[str]],
        ground_truth: Optional[Union[str, List[str]]] = None,
        target_neg: Optional[Union[str, List[str]]] = None,
        rephrase_prompts: Optional[Union[str, List[str]]] = None,
        locality_inputs:  Optional[Dict] = None,
        portability_inputs: Optional[Dict] = None,
        sequential_edit=False,
        verbose=True,
        **kwargs
    ):
        eval_metric= kwargs['eval_metric'] if 'eval_metric' in kwargs.keys() else 'exact match'
        test_generation = kwargs.pop('test_generation', False)

        assert len(prompts) == len(target_new)

        if hasattr(self.hparams, 'batch_size'):
            assert self.hparams.batch_size == 1, 'Single Editing: batch_size should be set to 1'
        
        if "requests" in kwargs.keys():
            requests = kwargs["requests"]
        else:
            requests = _prepare_requests(prompts, target_new, ground_truth, target_neg, rephrase_prompts, locality_inputs, portability_inputs, **kwargs)
        
        def text_generate(
            model,
            model_name,
            hparams: HyperParams,
            tok: AutoTokenizer,
            query,
            device,
            eval_metric: str = 'token_em',
            test_generation = False
        ):
            messages = [
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": query}
            ]
            text = self.tok.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True,
            )
            model_inputs = tok.encode(text, return_tensors="pt").to(f"cuda:{device}")
            template_length = len(model_inputs[0])
            generated_ids = model.generate(
                input_ids=model_inputs,
                max_new_tokens=512
            )
            trimmed_generated_ids = generated_ids[0][template_length:]
            response = tok.decode(trimmed_generated_ids, skip_special_tokens=True)
            return response

        all_results = []
        if 'pre_edit' in kwargs and kwargs['pre_edit'] is not None:
            results = kwargs['pre_edit']
            all_results = results
        else:
            for i, request in enumerate(tqdm(requests)):
                results = {}
                results['pre'] = {}
                results['pre']['rewrite_ans'] = text_generate(self.model, self.model_name, self.hparams, self.tok, request['prompt'], self.hparams.device, eval_metric=eval_metric, test_generation=test_generation)
                results['pre']['rephrase_ans'] = text_generate(self.model, self.model_name, self.hparams, self.tok, request['rephrase_prompt'], self.hparams.device, eval_metric=eval_metric, test_generation=test_generation)
                por_results = []
                for pr in request['portability']['por_hop']['prompt']:
                    por_results.append(text_generate(self.model, self.model_name, self.hparams, self.tok, pr, self.hparams.device, eval_metric=eval_metric, test_generation=test_generation))
                if 'locality' in request.keys() and 'loc_hop' in request['locality'].keys():
                    loc_results = []
                    for pr in request['locality']['loc_hop']['prompt']:
                        loc_results.append(text_generate(self.model, self.model_name, self.hparams, self.tok, pr, self.hparams.device, eval_metric=eval_metric, test_generation=test_generation))
                    results['pre']['locality_ans'] = loc_results
                results['pre']['portability_ans'] = por_results
                all_results.append(results)
            if 'pre_file' in kwargs and kwargs['pre_file'] is not None:
                json.dump(all_results, open(kwargs['pre_file'], 'w'), indent=4)

        def edit_func(request):
            if self.alg_name == 'IKE':
                edited_model, weights_copy, icl_examples = self.model, {}, self.apply_algo(
                    self.model,
                    self.tok,
                    [request],
                    self.hparams,
                    copy=False,
                    return_orig_weights=True,
                    keep_original_weight=False,
                    train_ds=kwargs['train_ds'] if self.alg_name == 'IKE' else None
                )
            else:
                edited_model, weights_copy = self.apply_algo(
                    self.model,
                    self.tok,
                    [request],
                    self.hparams,
                    copy=False,
                    return_orig_weights=True,
                    keep_original_weight=False,
                    train_ds=kwargs['train_ds'] if self.alg_name == 'IKE' else None
                )
                icl_examples = None
            return edited_model, weights_copy, icl_examples
        
        def post_edit_results(all_results, request, edited_model, idx, eval_metric, test_generation, icl_examples, **kwargs):
            if self.alg_name == 'IKE':
                all_results[idx].update({
                    'case_id': idx,
                    "requested_rewrite": request,
                    "post": compute_icl_edit_quality(self.model, self.model_name, self.hparams, self.tok, icl_examples, request, self.hparams.device),
                })
            else:
                results_post = {}
                results_post['rewrite_ans'] = text_generate(edited_model, self.model_name, self.hparams, self.tok, request['prompt'], self.hparams.device, eval_metric=eval_metric, test_generation=test_generation)
                results_post['rephrase_ans'] = text_generate(edited_model, self.model_name, self.hparams, self.tok, request['rephrase_prompt'], self.hparams.device, eval_metric=eval_metric, test_generation=test_generation)
                por_results = []
                for pr in request['portability']['por_hop']['prompt']:
                    por_results.append(text_generate(edited_model, self.model_name, self.hparams, self.tok, pr, self.hparams.device, eval_metric=eval_metric, test_generation=test_generation))
                if 'locality' in request.keys() and 'loc_hop' in request['locality'].keys():
                    loc_results = []
                    for pr in request['locality']['loc_hop']['prompt']:
                        loc_results.append(text_generate(edited_model, self.model_name, self.hparams, self.tok, pr, self.hparams.device, eval_metric=eval_metric, test_generation=test_generation))
                    results_post['locality_ans'] = loc_results
                results_post['portability_ans'] = por_results
                if test_generation:
                    if self.hparams.alg_name == 'GRACE':
                        results_post['fluency'] = test_generation_quality(model=edited_model,tok=self.tok,prefixes=request['prompt'] if isinstance(request['prompt'],list) else [request['prompt'],], max_out_len=100, vanilla_generation=True)
                    else:
                        results_post['fluency'] = test_generation_quality(model=edited_model,tok=self.tok,prefixes=request['prompt'] if isinstance(request['prompt'],list) else [request['prompt'],], max_out_len=100, vanilla_generation=False)
                all_results[idx].update({
                    'case_id': idx,
                    "requested_rewrite": request,
                    "post": results_post
                })
            if verbose:
                LOG.info(f"{idx} editing: {request['prompt']} -> {request['target_new']}")

        if sequential_edit:
            for i, request in enumerate(tqdm(requests, total=len(requests))):
                edited_model, weights_copy, icl_examples = edit_func(request)
            for i, request in enumerate(requests):
                post_edit_results(all_results, request, edited_model, i, eval_metric, test_generation, icl_examples, **kwargs)
        else:
            for i, request in enumerate(tqdm(requests, total=len(requests))):
                edited_model, weights_copy, icl_examples = edit_func(request)
                post_edit_results(all_results, request, edited_model, i, eval_metric, test_generation, icl_examples, **kwargs)
                if self.alg_name == 'KN' or self.alg_name == 'GRACE' or self.alg_name == 'WISE':
                    with torch.no_grad():
                        weights_copy()
                elif self.alg_name == 'LoRA' or self.alg_name == 'QLoRA' or self.alg_name == 'DPO':
                    edited_model.unload()
                    del self.model.peft_config
                elif self.alg_name == 'MELO':
                    self.model = edited_model
                elif self.alg_name == 'LoRA' or self.alg_name == 'QLoRA' or self.alg_name == 'DPO':
                    self.model = edited_model
                else:
                    with torch.no_grad():
                        for k, v in weights_copy.items():
                            nethook.get_parameter(self.model, k)[...] = v.to(f"cuda:{self.hparams.device}")

        if isinstance(edited_model, LORA):
            edited_model = edited_model.model
        if len(all_results) != 0:
            summary_metrics(all_results)

        return all_results, edited_model, weights_copy

    def deep_edit(
        self,
        datasets
    ):
        metrics = self.apply_algo(datasets, self.hparams)
        return metrics


In [ ]:
# Load Dataset

# %mkdir data
!huggingface-cli download zjunlp/KnowEdit --repo-type dataset --local-dir /kaggle/working/EasyEdit/data


Fetching 14 files:   0%|                                 | 0/14 [00:00<?, ?it/s]Downloading 'benchmark/WikiBio/wikibio-train-all.json' to '/kaggle/working/EasyEdit/data/.cache/huggingface/download/benchmark/WikiBio/8rs6DLxZ_QZwK6fLwCiUot6Qc1I=.fd11768426fe24820d7b99e11acd4db7e2277721.incomplete'

blender_train.json:   0%|                           | 0.00/24.0M [00:00<?, ?B/s]Downloading 'benchmark/Convsent/blender_test.json' to '/kaggle/working/EasyEdit/data/.cache/huggingface/download/benchmark/Convsent/uG92iOpRw2ySGVaHQtkdFcCzJmY=.cf80f02b519e2fd910c59b216aa8bc81a50d8b18.incomplete'


wikibio-train-all.json:   0%|                        | 0.00/454k [00:00<?, ?B/s]


README.md: 100%|███████████████████████████| 23.8k/23.8k [00:00<00:00, 85.7MB/s]
Download complete. Moving file to /kaggle/working/EasyEdit/data/README.md



wikibio-test-all.json:   0%|                         | 0.00/310k [00:00<?, ?B/s]



blender_val.json:   0%|                             | 0.00/1.33M [00:00<?, ?B/s]


In [ ]:
EDITING_METHOD = 'MELO' # FT, ICE, LoRA, MELO
is_ice = False
if EDITING_METHOD == 'ICE':
    is_ice = True

if EDITING_METHOD != 'KGEdit':
    if EDITING_METHOD == 'ICE':
        EDITING_METHOD = 'IKE'
        is_ice = True
    
    if MODEL == 'gpt2-xl':
        with open(f'/kaggle/working/EasyEdit/hparams/{EDITING_METHOD}/gpt2-xl.yaml', 'r', encoding='utf-8') as f:
            config = f.read()
        
        if EDITING_METHOD == 'SERAC':
            config = re.sub(r'(?<=model_name: )\./hugging_cache', 'openai-community', config)
            config = re.sub(r'(?<=small_name: )\./hugging_cache', 'openai-community', config)
            config = re.sub(r'(?<=cls_name: )\./hugging_cache', 'distilbert', config)
            config = re.sub(r'(?<=archive: )\./results/models/SERAC/gpt2-xl', '/kaggle/working/EasyEdit/results/models/SERAC/gpt2-xl_bak', config)
            
        elif EDITING_METHOD == 'MEND':
            with open(f'/kaggle/working/EasyEdit/hparams/TRAINING/{EDITING_METHOD}/gpt2-xl.yaml', 'r', encoding='utf-8') as tf:
                training_config = tf.read()
            config = re.sub(r'(?<=model_name: )\./hugging_cache', 'openai-community', config)
            training_config = re.sub(r'(?<=model_name: )\./hugging_cache', 'openai-community', training_config)
            training_config = re.sub(r'(?<=tokenizer_name: )\./hugging_cache', 'openai-community', training_config)
            config = re.sub(r'(?<=archive: )\./results/models/MEND/gpt2-xl', '/kaggle/working/EasyEdit/results/models/MEND/gpt2-xl_bak', config)
        
            with open(f'/kaggle/working/EasyEdit/hparams/TRAINING/{EDITING_METHOD}/gpt2-xl.yaml', 'w', encoding='utf-8') as tf:
                tf.write(training_config)
        
        elif EDITING_METHOD == 'MELO':
            with open(f'/kaggle/working/EasyEdit/hparams/{EDITING_METHOD}/gpt2-xl.yaml', 'r', encoding='utf-8') as f:
                config = f.read()
            config = re.sub(r'\./hugging_cache/gpt2-xl', 'openai-community/gpt2-xl', config)
            config = re.sub(r'\./hugging_cache/distilbert-base-cased', 'distilbert/distilbert-base-cased', config)
            
        else:
            config = re.sub(r'(?<=model_name: ").+(?=/gpt2-xl")', 'openai-community', config)
            
    elif 'qwen' in MODEL:
        model_size = re.search(r'(?<=-)(\d\.)?\db', MODEL).group(0)
        model_type = '-instruct'
        if model_type not in MODEL:
            model_type = ''
        if EDITING_METHOD == 'MELO':
            with open(f'/kaggle/working/EasyEdit/hparams/{EDITING_METHOD}/gpt2-xl.yaml', 'r', encoding='utf-8') as f:
                config = f.read()
            config = re.sub(r'\./hugging_cache/gpt2-xl', f'/kaggle/input/qwen2.5/transformers/{model_size}{model_type}/1', config)
            config = re.sub(r'\./hugging_cache/distilbert-base-cased', 'distilbert/distilbert-base-cased', config)
            config = re.sub(r'(?<=class_name: ).+(?=\n)', 'Qwen2ForCausalLM', config)
            config = re.sub(r'(?<=tokenizer_class: ).+(?=\n)', 'Qwen2Tokenizer', config)
            config = re.sub(r'(?<=grace_layer: )transformer.h.35.mlp.c_fc', 'model.layers.8.mlp.down_proj', config)
            config = re.sub(r'transformer.h.36.mlp.c_fc', 'model.layers.10.mlp.gate_proj\n    - model.layers.10.mlp.up_proj', config)
            config = re.sub(r'transformer.h.37.mlp.c_fc', 'model.layers.11.mlp.gate_proj\n    - model.layers.11.mlp.up_proj', config)
        else:
            with open(f'/kaggle/working/EasyEdit/hparams/{EDITING_METHOD}/qwen2.5-7b.yaml', 'r', encoding='utf-8') as f:
                config = f.read()
            config = re.sub(r'(?<=\nmodel_name: ").+(?=")', f'/kaggle/input/qwen2.5/transformers/{model_size}{model_type}/1', config)
        
    config = re.sub(r'(?<=device: )\d+', '0', config)
    config = re.sub(r'(?<=batch_size: )\d+', '1', config)
    
    if is_ice is True:
        EDITING_METHOD = 'ICE'
        os.makedirs(f'/kaggle/working/EasyEdit/hparams/{EDITING_METHOD}', exist_ok=True)
    
    with open(f'/kaggle/working/EasyEdit/hparams/{EDITING_METHOD}/{MODEL}.yaml', 'w', encoding='utf-8') as f:
        f.write(config)


In [ ]:
print(config)


In [ ]:
def eval(result_path: str):
    if not path.exists(result_path):
        raise FileNotFoundError(f'File {result_path} not found!')
    
    with open(result_path,'r') as file:
        datas=json.load(file)
    #data_rome_counterfact['post'].keys()  dict_keys(['rewrite_acc', 'locality', 'portability'])
    Edit_Succ_list=[data_rome_counterfact['post']['rewrite_acc'][0] for data_rome_counterfact in datas]
    Edit_Succ=sum(Edit_Succ_list)/len(Edit_Succ_list)*100
    print('Edit_Succ:',Edit_Succ)
    
    Portability_list=[]
    for data_rome_counterfact in datas:
        case_list=[]
        for key in data_rome_counterfact['post']['portability'].keys():
            case_list.append(sum(data_rome_counterfact['post']['portability'][key])/len(data_rome_counterfact['post']['portability'][key])*100)
        if len(case_list) != 0:
            Portability_list.append(np.mean(case_list))
    Overall_portability = np.mean(Portability_list)
    print('Overall_portability:',Overall_portability)

    Locality_list=[]
    for data_rome_counterfact in datas:
        case_list=[]
        for key in data_rome_counterfact['post']['locality'].keys():
            case_list.append(sum(data_rome_counterfact['post']['locality'][key])/len(data_rome_counterfact['post']['locality'][key])*100)
        if len(case_list) != 0:
            Locality_list.append(np.mean(case_list))
    Overall_locality = np.mean(Locality_list)
    print('Overall_locality:',Overall_locality)
    
    Fluency_list=[x['post']['fluency']['ngram_entropy'] for x in datas]
    Fluency=sum(Fluency_list)/len(Fluency_list)*100
    print('Fluency:',Fluency)


In [ ]:
def run_edit(
    hparams_file: str,
    data_dir: str,
    editing_method: str = 'ROME',
    ds_size=None,
    metrics_save_dir: str = './output',
    datatype=None, # counterfact, recent, zsre or wikibio
    pre_file=None, #pre-edit file name
    train_data_path=None, # Used for IKE
    use_chat_template=False
):
        
    print('--Knowledge Editing Pipeline started...--')
    editing_methods = {
        'FT': FTHyperParams,
        'ICE': IKEHyperParams,
        'LoRA': LoRAHyperParams,
        'MELO': MELOHyperParams,
    }
    if editing_method not in editing_methods:
        raise NotImplementedError('Unknown editing method! The following methods are supported:\n' + ';\n'.join([method for method in editing_methods]))
    
    editing_hparams = editing_methods[editing_method]
    if EDITING_METHOD == 'MEND':
        print('--Additional Training for MEND--')
        training_hparams = MENDTrainingHparams.from_hparams('/kaggle/working/EasyEdit/hparams/TRAINING/MEND/gpt2-xl.yaml')
        train_ds = ZsreDataset('/kaggle/working/EasyEdit/data/zsre/zsre_mend_train_10000.json', config=training_hparams)
        eval_ds = ZsreDataset('/kaggle/working/EasyEdit/data/zsre/zsre_mend_eval.json', config=training_hparams)

        trainer = EditTrainer(
            config=training_hparams,
            train_set=train_ds,
            val_set=eval_ds
        )

        trainer.run()

    print('--Initialising dataset--')

    datas = KnowEditDataset(data_dir, size=ds_size)
    
    if datatype == 'counterfact' or datatype == 'recent' or datatype == 'zsre':
        prompts=[data['prompt'] for data in datas]
        subjects=[data['subject'] for data in datas]
        target_new = [data['target_new'] for data in datas]
        
        portability_r =[data['portability_r'] for data in datas]
        portability_s =[data['portability_s'] for data in datas]
        portability_l =[data['portability_l'] for data in datas]

        portability_reasoning_prompts=[]
        portability_reasoning_ans=[]
        portability_Logical_Generalization_prompts=[]
        portability_Logical_Generalization_ans=[]
        portability_Subject_Aliasing_prompts=[]
        portability_Subject_Aliasing_ans=[]
        
        portability_data = [portability_r,portability_s,portability_l]
        portability_prompts = [portability_reasoning_prompts,portability_Subject_Aliasing_prompts,portability_Logical_Generalization_prompts]
        portability_answers = [portability_reasoning_ans,portability_Subject_Aliasing_ans,portability_Logical_Generalization_ans]
        
        for data, portable_prompts, portable_answers in zip(portability_data,portability_prompts,portability_answers):
            for item in data:
                if item is None:
                    portable_prompts.append(None)
                    portable_answers.append(None)
                else:
                    temp_prompts = []
                    temp_answers = []
                    for pr in item:
                        prompt=pr["prompt"]
                        an=pr["ground_truth"]
                        while isinstance(an,list):
                            an = an[0]
                        if an.strip() =="":
                            continue
                        temp_prompts.append(prompt)
                        temp_answers.append(an)
                    portable_prompts.append(temp_prompts)
                    portable_answers.append(temp_answers)
        assert len(prompts) == len(portability_reasoning_prompts) == len(portability_Logical_Generalization_prompts) == len(portability_Subject_Aliasing_prompts)
        
        locality_rs = [data['locality_rs'] for data in datas]
        locality_f = [data['locality_f'] for data in datas]
        locality_Relation_Specificity_prompts=[]
        locality_Relation_Specificity_ans=[]
        locality_Forgetfulness_prompts=[]        
        locality_Forgetfulness_ans=[]
        
        locality_data = [locality_rs, locality_f]
        locality_prompts = [locality_Relation_Specificity_prompts,locality_Forgetfulness_prompts]
        locality_answers = [locality_Relation_Specificity_ans,locality_Forgetfulness_ans]
        
        for data, local_prompts, local_answers in zip(locality_data,locality_prompts,locality_answers):
            for item in data:
                if item is None:
                    local_prompts.append(None)
                    local_answers.append(None)
                else:
                    temp_prompts = []
                    temp_answers = []
                    for pr in item:
                        prompt=pr["prompt"]
                        an=pr["ground_truth"]
                        while isinstance(an,list):
                            an = an[0]
                        if an.strip() =="":
                            continue
                        temp_prompts.append(prompt)
                        temp_answers.append(an)
                    local_prompts.append(temp_prompts)
                    local_answers.append(temp_answers)
        assert len(prompts) == len(locality_Relation_Specificity_prompts) == len(locality_Forgetfulness_prompts)
        
        locality_inputs = {}
        portability_inputs = {}
        
        locality_inputs = {
            'Relation_Specificity':{
                'prompt': locality_Relation_Specificity_prompts,
                'ground_truth': locality_Relation_Specificity_ans
            },
            'Forgetfulness':{
                'prompt': locality_Forgetfulness_prompts,
                'ground_truth': locality_Forgetfulness_ans
            }
        }
        portability_inputs = {
            'Subject_Aliasing':{
                'prompt': portability_Subject_Aliasing_prompts,
                'ground_truth': portability_Subject_Aliasing_ans
            },
            'reasoning':{
                'prompt': portability_reasoning_prompts,
                'ground_truth': portability_reasoning_ans           
            },
            'Logical_Generalization':{
                'prompt': portability_Logical_Generalization_prompts,
                'ground_truth': portability_Logical_Generalization_ans           
            }
        }

    if datatype == 'wikibio':
        prompts=[data['prompt'] for data in datas]
        subjects=[data['subject'] for data in datas]
        target_new = [data['target_new'] for data in datas]
        
        locality_rs = [data['locality_rs'] for data in datas]
        locality_f = [data['locality_f'] for data in datas]
        locality_Relation_Specificity_prompts=[]
        locality_Relation_Specificity_ans=[]
        
        locality_data = [locality_rs]
        locality_prompts = [locality_Relation_Specificity_prompts]
        locality_answers = [locality_Relation_Specificity_ans]
        for data, local_prompts, local_answers in zip(locality_data,locality_prompts,locality_answers):
            for item in data:
                if item is None:
                    local_prompts.append(None)
                    local_answers.append(None)
                else:
                    temp_prompts = []
                    temp_answers = []
                    for pr in item:
                        prompt=pr["prompt"]
                        an=pr["ground_truth"]
                        while isinstance(an,list):
                            an = an[0]
                        if an.strip() =="":
                            continue
                        temp_prompts.append(prompt)
                        temp_answers.append(an)
                    local_prompts.append(temp_prompts)
                    local_answers.append(temp_answers)
        assert len(prompts) == len(locality_Relation_Specificity_prompts)
        portability_inputs = None
        locality_inputs = {}
        locality_inputs = {
            'Relation_Specificity':{
                'prompt': locality_Relation_Specificity_prompts,
                'ground_truth': locality_Relation_Specificity_ans
            }
        }

    print('--Dataset initialised. Collecting editing hyperparameters--')

    hparams = None
    if editing_method != 'KGEdit':
        hparams = editing_hparams.from_hparams(hparams_file)
        
    if pre_file is None:
        if editing_method != 'KGEdit':
            pre_file = f"./{hparams.model_name.split('/')[-1]}_{datatype}_pre_edit.json"
        else:
            pre_file = f"./{MODEL}_{datatype}_pre_edit.json"
    print(f'pre edit file: {pre_file}')

    if pre_file is not None and os.path.exists(pre_file):
        with open(pre_file,'r', encoding='utf-8') as pre_f:
            pre_edit = json.load(pre_f)
        assert len(pre_edit) == len(prompts)
    else:
        pre_edit = None

    if editing_method == 'IKE':
        train_ds = KnowEditDataset(train_data_path)
        sentence_model = SentenceTransformer(hparams.sentence_model_name).to(f'cuda:{hparams.device}')
        encode_ike_facts(sentence_model, train_ds, hparams)
    elif editing_method == 'ICE':
        hparams.use_icl_examples = False
        train_ds = None
    else:
        train_ds = None
    print('--Preparation complete. Running Knowledge Editing--')
    
    if editing_method == 'KGEdit':
        editor = KGEditor(hparams, use_nli=use_nli, use_chat_template=use_chat_template)
        # metrics, edited_model = editor.edit_requests(
        # requests=requests, 
        # **kwargs
        # )
    else:
        editor = BaseEditor.from_hparams(hparams)
    metrics, edited_model, _ = editor.edit(
        prompts=prompts,
        target_new=target_new,
        subject=subjects,
        locality_inputs=locality_inputs,
        portability_inputs=portability_inputs,
        train_ds=train_ds,
        keep_original_weight=True,
        pre_file=pre_file,
        pre_edit=pre_edit,
        test_generation=True,
    )
    print('--Knowledge edited successfully. Saving results--')

    if not os.path.exists(metrics_save_dir):
        os.makedirs(metrics_save_dir)
    result_path = os.path.join(metrics_save_dir, f'{editing_method}_{datatype}_{hparams.model_name.split("/")[-1]}_results.json')
    with open(result_path, 'w', encoding='utf-8') as res_f:
        json.dump(metrics, res_f, indent=4)
    print('--Saving complete.\tEvaluating results--')
    eval(result_path)


In [ ]:
SAMPLE = 'recent'
SAMPLE_TYPE = 'train'

train_data_dirs = {
    'wikibio': '/kaggle/working/EasyEdit/data/benchmark/WikiBio/wikibio-train-all.json', # OK
    'counterfact': '/kaggle/working/EasyEdit/data/benchmark/wiki_counterfact/train_cf.json', # OK
    'recent': '/kaggle/working/EasyEdit/data/benchmark/wiki_recent/recent_train.json', # OK
    'zsre': None
}
test_data_dirs = {
    'wikibio': '/kaggle/working/EasyEdit/data/benchmark/WikiBio/wikibio-test-all.json', # OK
    'counterfact': '/kaggle/working/EasyEdit/data/benchmark/wiki_counterfact/test_cf.json', # OK
    'recent': '/kaggle/working/EasyEdit/data/benchmark/wiki_recent/recent_test.json', # OK
    'zsre': '/kaggle/working/EasyEdit/data/benchmark/ZsRE/ZsRE-test-all.json'
}

if SAMPLE_TYPE == 'train':
    sample_path = train_data_dirs[SAMPLE]
elif SAMPLE_TYPE == 'test':
    sample_path = test_data_dirs[SAMPLE]

hparams_file_path = f'/kaggle/working/EasyEdit/hparams/{EDITING_METHOD}/{MODEL}.yaml'

run_edit(
    hparams_file=hparams_file_path,
    data_dir=sample_path,
    editing_method=EDITING_METHOD,
    ds_size=50,
    datatype=SAMPLE,
    use_chat_template=USE_CHAT_TEMPLATE
)


--Knowledge Editing Pipeline started...--
--Initialising dataset--
--Dataset initialised. Collecting editing hyperparameters--


2025-06-24 00:52:11,278 - __main__ - INFO - Instantiating model


pre edit file: ./1_recent_pre_edit.json
--Preparation complete. Running Knowledge Editing--
We are creating the logger files


2025-06-24 00:52:26,678 - __main__ - INFO - AutoRegressive Model detected, set the padding side of Tokenizer to left...
  0%|          | 0/50 [00:00<?, ?it/s]/kaggle/working/EasyEdit/easyeditor/models/melo/peft_egg/src/peft/tuners/melo.py:254: UserWarning: fan_in_fan_out is set to True but the target module is `torch.nn.Linear`. Setting fan_in_fan_out to False.
  warnings.warn(


Target Grace Layer is found: model.layers.8.mlp.down_proj


...

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

2025-06-24 00:55:25,282 - __main__ - INFO - 0 editing: The gender of Ingrīda Amantova is -> female  

 {'pre': {'rewrite_acc': [1.0], 'portability': {}, 'fluency': {'ngram_entropy': 6.23880373256265}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'The gender of Ingrīda Amantova is', 'target_new': 'female', 'ground_truth': '<|endoftext|>', 'portability': {}, 'locality': {'Relation_Specificity': {'prompt': ['The place of birth of Ingrīda Amantova is', 'The name of the country of citizenship of Ingrīda Amantova is', 'The name of the alma mater of Ingrīda Amantova is', 'The occupation of Ingrīda Amantova is', 'The name of the award Ingrīda Amantova won is'], 'ground_truth': ['Cēsis', 'Soviet Union', 'University of Latvia', 'luger', 'Cross of Recognition']}}, 'subject': 'Ingrīda Amantova'}, 'post': {'rewrite_acc': [1.0], 'locality': {'Relation_Specificity_acc': [1.0, 1.0, 1.0, 1.0, 1.0]}, 'portability': {}, 'fluency': {'ngram_entropy': 6.198130696140149}}}
  2%|▏         | 1/50 [00:12<10:2

Metrics Summary:  {'pre': {'rewrite_acc': 0.3207965367965368}, 'post': {'rewrite_acc': 1.0, 'locality': {'Forgetfulness_acc': 1.0, 'Relation_Specificity_acc': 1.0}}}
--Knowledge edited successfully. Saving results--
--Saving complete.	Evaluating results--
Edit_Succ: 100.0
Overall_portability: 40.49107142857143
Overall_locality: 100.0
Fluency: 612.2117935597008
